# **DataSheet Description**

---


En este proyecto emplearemos un dataset que contiene información de los resultados de las carreras de el panorama nacional de carreras de caballos.

- **Puesto** (Clase): Posición en la que terminan la carrera **int64**
- **NombreCaballo**: Nombre de caballo  **object**
- **Peso** : Peso de caballo  **object**
- **Edad**: Edad de caballo **int64**
- **DistanciaMeta**: Distancia respecto a ganador en llegada a meta **object** (DROPEAR)
- **Mantilla**: Dorsal   **int64**
- **Dividendo** : Pago por euro apostado por ganador de carrera **object** (DROPEAR)
- **Propietario** : Cuadra Propietaria **object**
- **Preparador**: Nombre Preparador **object**
- **Jinete**: Jinete o Amazona que monta al caballo en esta carrera  **object**
- **Problemas**: Problemas que posee en esta carrera el caballo  **object**
- **Fecha**: Fecha Carrera   **object**
- **Hora**: Hora carrera  **object**
- **Terreno**: Estado del terreno en carrera **object**
- **Distancia**: Longitud carrera  **float64**
- **Tipo**: Tipo de terreno en carrera  **object**
- **SentidoHipodromo**:  (0 -> Sentido Normal, 1 -> Inverso) **int64**
- **Meteorología**: Previsión estado meteorología del día entero **object** (Tiempo3)
- **Lluvia**: Previsión cantidad de lluvia del dia entero  **object**
- **Viento**: Previsión velocidad de viento del dia entero  **int64**
- **Temperatura**: Previsión de temp. del horario de la carrera  **int64**
- **Hipodromo**: Nombre del hipódromo de la carrera   **object**

In [16]:
import pandas as pd
import numpy as np
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import sklearn.compose
from xgboost import XGBClassifier
from sklearn.metrics import f1_score

url = 'https://raw.githubusercontent.com/SergioNietoP/ML-Try-Catch/main/Learning%20Datasheets/Turf%20Prediction/Data%20Turf%20Races%20Train%20-%2030%20-%201006.csv'

url1 = 'https://raw.githubusercontent.com/SergioNietoP/ML-Try-Catch/main/Learning%20Datasheets/Turf%20Prediction/Data%20Turf%20Races%20Test%20-%2023%20-%201706.csv'
test = pd.read_csv(url1, sep=',')

# test_labels = test.pop('Puesto')
# y_testNN = test_labels


train = pd.read_csv(url, sep=',')


ogData = train
train.head()
train.count()



Puesto              2764
NombreCaballo       2764
Peso                2764
Edad                2764
DistanciaMeta       2363
Mantilla            2764
Dividendo           2749
Propietario         2764
Preparador          2764
Jinete              2763
Problemas           1481
Fecha               2764
Hora                2764
Terreno             2764
Distancia           2764
Tipo                2763
SentidoHipodromo    2764
Meteorología        2764
LLuvia              2764
Viento              2764
Temperatura         2764
Hipodromo           2764
dtype: int64

In [17]:
print(train['DistanciaMeta'].unique())

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2764 entries, 0 to 2763
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Puesto            2764 non-null   int64  
 1   NombreCaballo     2764 non-null   object 
 2   Peso              2764 non-null   object 
 3   Edad              2764 non-null   int64  
 4   DistanciaMeta     2363 non-null   object 
 5   Mantilla          2764 non-null   int64  
 6   Dividendo         2749 non-null   object 
 7   Propietario       2764 non-null   object 
 8   Preparador        2764 non-null   object 
 9   Jinete            2763 non-null   object 
 10  Problemas         1481 non-null   object 
 11  Fecha             2764 non-null   object 
 12  Hora              2764 non-null   object 
 13  Terreno           2764 non-null   object 
 14  Distancia         2764 non-null   int64  
 15  Tipo              2763 non-null   object 
 16  SentidoHipodromo  2764 non-null   int64  


In [15]:
def comaDividendo():
    for i in range(len(train.index)):
      train.at[i, 'Dividendo'] = str(train.at[i, 'Dividendo']).replace(',', '.')

comaDividendo()

In [ ]:
# train = train.drop('Dividendo', axis=1)

def dividendToFloat():
  for index, row in train.iterrows():
    train.at[index, 'Dividendo'] = train.at[index, 'Dividendo'].replace('€', '')
    if( '-' in train.at[index, 'Dividendo']):
      train.at[index, 'Dividendo'] = '0.0'


dividendToFloat()
train['Dividendo'] = train['Dividendo'].astype(float)


In [ ]:
train.info()

# Preprocesamiento
---

In [ ]:
train['PuestoOg'] = train['Puesto']

In [ ]:
pd.set_option('display.max_columns', None)
train.head(10)


## **Convertir DistanciaMeta (Test & Train)**

In [ ]:
train["DistanciaMeta"] = train["DistanciaMeta"].fillna(0)

In [ ]:
def conversionDistanciaMetaTrain():
  train['DistanciaMeta'] = train['DistanciaMeta'].astype('str')
  for i in range(len(train.index)):
    decimal = 0
    if('LEJOS' in train.at[i, 'DistanciaMeta'] or 'Lejos' in train.at[i, 'DistanciaMeta'] or 'lejos' in train.at[i, 'DistanciaMeta']):
      train.at[i, 'DistanciaMeta'] = '20'
    if('DISTANCIADO' in train.at[i, 'DistanciaMeta'] or'Distanciado' in train.at[i, 'DistanciaMeta'] or 'distanciado' in train.at[i, 'DistanciaMeta']):
      train.at[i, 'DistanciaMeta'] = '20'
    if('CUELLO' in train.at[i, 'DistanciaMeta'] or 'Cuello' in train.at[i, 'DistanciaMeta'] or 'cuello' in train.at[i, 'DistanciaMeta']):
      train.at[i, 'DistanciaMeta'] = '0.3'
    if(('CABEZA') in train.at[i, 'DistanciaMeta'] or ('Cabeza') in train.at[i, 'DistanciaMeta'] or ('cabeza') in train.at[i, 'DistanciaMeta']):
      train.at[i, 'DistanciaMeta'] = '0.2'
    if(('NARIZ') in train.at[i, 'DistanciaMeta'] or ('Nariz') in train.at[i, 'DistanciaMeta'] or ('nariz') in train.at[i, 'DistanciaMeta']):
      train.at[i, 'DistanciaMeta'] = '0.1'
    if(('EMPATE') in train.at[i, 'DistanciaMeta'] or ('Empate') in train.at[i, 'DistanciaMeta'] or ('empate') in train.at[i, 'DistanciaMeta'] or train.at[i, 'DistanciaMeta'] == '-'):
      train.at[i, 'DistanciaMeta'] = '0'
    if(('CABALLO') in train.at[i, 'DistanciaMeta'] or ('Caballo') in train.at[i, 'DistanciaMeta'] or ('caballo') in train.at[i, 'DistanciaMeta']):
      train.at[i, 'DistanciaMeta'] = '1'
    if((' C') in train.at[i, 'DistanciaMeta']):
      train.at[i, 'DistanciaMeta'] = train.at[i, 'DistanciaMeta'].replace(' C', '')
    if(train.at[i, 'DistanciaMeta'] == '1/4'):
      train.at[i, 'DistanciaMeta'] = '0.25'
    if(train.at[i, 'DistanciaMeta'] == '1/2'):
      train.at[i, 'DistanciaMeta'] = '0.5'
    if(train.at[i, 'DistanciaMeta'] == '3/4'):
      train.at[i, 'DistanciaMeta'] = '0.75'
    if((' 1/4') in train.at[i, 'DistanciaMeta']):
      train.at[i, 'DistanciaMeta'] = train.at[i, 'DistanciaMeta'].replace(' 1/4', '')
      train.at[i, 'DistanciaMeta'] = train.at[i, 'DistanciaMeta'] + '.25'
    if((' 1/2') in train.at[i, 'DistanciaMeta']):
      train.at[i, 'DistanciaMeta'] = train.at[i, 'DistanciaMeta'].replace(' 1/2', '')
      train.at[i, 'DistanciaMeta'] = train.at[i, 'DistanciaMeta'] + '.5'
    if((' 3/4') in train.at[i, 'DistanciaMeta']):
      train.at[i, 'DistanciaMeta'] = train.at[i, 'DistanciaMeta'].replace(' 3/4', '')
      train.at[i, 'DistanciaMeta'] = train.at[i, 'DistanciaMeta'] + '.75'

conversionDistanciaMetaTrain();
train['DistanciaMeta'] = train['DistanciaMeta'].astype('float')
train['DistanciaMeta'] = np.where(train['DistanciaMeta'] > 20, 20, train['DistanciaMeta'])
train['DistanciaMeta'].describe()


## **Tratamiento (Puesto)(Train)**

In [ ]:
train = train.drop(train[train.Puesto >= 90].index)
train = train.reset_index(drop=True)
train.count()
# train['Puesto'] = train['Puesto'].replace(95, 10) # Penalización Parado

## **Tratamiento (Fecha)**


In [ ]:
import datetime as dt
from datetime import date


def switchMonth(month):
  if month == 'ENERO':
   return '01'
  elif month == 'FEBRERO':
    return '02'
  elif month == 'MARZO':
   return '03'
  elif month == 'ABRIL':
    return '04'
  elif month == 'MAYO':
    return '05'
  elif month == 'JUNIO':
    return '06'
  elif month == 'JULIO':
    return '07'
  elif month == 'AGOSTO':
    return '08'
  elif month == 'SEPTIEMBRE':
    return '09'
  elif month == 'OCTUBRE':
     return '10'
  elif month == 'NOVIEMBRE':
    return '11'
  elif month == 'DICIEMBRE':
    return '12'

def convertDate(fechas):
  index = 0
  for element in fechas:
    sum = 0
    year = element[-4:]
    dateAux = year
    day = element[:2]
    if(int(day) < 10):
      day = "0" + day[:1]
    month = element[5:]
    month = ''.join(month.split())[:-6].upper()
    month = switchMonth(month)
    dateAux = dateAux + "-" + month + "-" + day

    today = date.today()
    yearToday = int(str(today)[:4])
    monthToday = int(str(today)[5:7])
    dayToday = int(str(today)[8:10])
    if(yearToday > int(year)):
      sum += (yearToday - int(year))*365
    sum += (monthToday - int(month))*30
    sum += (dayToday - int(day))
    datetimeAux = dt.datetime(int(year), int(month), int(day))
    train.at[index, 'FechaAux'] = datetimeAux

    train.at[index, 'year'] = int(year)
    train.at[index, 'month'] = int(month)
    train.at[index, 'day'] = int(day)
    calculateSeason(int(month), index)

    # train.at[index, 'DiasDesdeCarrera'] = sum
    index += 1

def calculateSeason(month, index):
  # train['Invierno'] = 0
  # train['Primavera'] = 0
  # train['Verano'] = 0
  train['Otoño'] = 0
  # if(month >= 12 and month <=2):
  #    train.at[index, 'Invierno'] = month
  # if(month >= 3 and month <=5):
  #    train.at[index, 'Primavera'] = month
  # if(month >= 6 and month <=8):
  #    train.at[index, 'Verano'] = month
  if(month >= 9 and month <=11):
     train.at[index, 'Otoño'] = month




t_array = train["Fecha"]
convertDate(t_array)
train['FechaAux'].head(120)

In [ ]:
train.head(5)

In [ ]:
from datetime import datetime

def daysSincePreviousRace():
  for index, row in train.iterrows():

    nombreCaballo = row['NombreCaballo']
    dict = {}
    conta = 0
    for indice, fila in train.iterrows():
      if( fila['NombreCaballo'] == nombreCaballo):
        rowData = fila['FechaAux'].date()
        dict[rowData] = indice
        conta += 1

    listaFechasAux = list(filter(lambda n: n < row['FechaAux'].date(), list(dict.keys())))
    if(listaFechasAux):
      mostRecentRace = max(listaFechasAux)
      jockeyRecent = train.at[dict[mostRecentRace], 'Jinete']
      daysDifference = row['FechaAux'].date() - mostRecentRace
      train.at[index, 'DaysSincePreviousRace'] = daysDifference.days


      if(jockeyRecent == row['Jinete']):
         train.at[index, 'SameJockey'] = 1
      else:
        train.at[index, 'SameJockey'] = 0
    else:
        train.at[index, 'SameJockey'] = 0
        train.at[index, 'DaysSincePreviousRace'] = 45



    # Rellenar dictionario con keys que sean las fechas en las que ha participado el caballo y los values , el index en el que aparecen
    # Deberias buscar primero las fechas anteriores en las que corrio un caballo antes de este codigo
    # y en caso de que no haya fechas hacer algo al respecto
    # En caso de qie si haya , por keys.values y coger anteriores y la mas reciente. Obtener key.value de jinete y comprobar con jinete actual .
    # Rambien calcular dias de diferencia desde fecha


daysSincePreviousRace()
train.info()

In [ ]:
train.head()

## **Tratamiento (NombreCaballo)(Train)**

In [ ]:
test['Propietario'] = test['Propietario'].str.strip()
test['NombreCaballo'] = test['NombreCaballo'].str.strip()
test['Jinete'] = test['Jinete'].str.strip()
test['Preparador'] = test['Preparador'].str.strip()

test['Jinete'] = test['Jinete'].replace('. ', '.')
test['Preparador'] = test['Preparador'].replace('. ', '.')
test['Propietario'] = test['Propietario'].replace('. ', '.')

def removePointSS():
  for index, fila in test.iterrows():
      test.at[index,'Jinete'] = test.at[index, 'Jinete'].replace('. ', '.')
      test.at[index,'Preparador'] = test.at[index, 'Preparador'].replace('. ', '.')
      test.at[index, 'Propietario'] = test.at[index, 'Propietario'].replace('. ', '.')
  for index, fila in train.iterrows():
      train.at[index,'Jinete'] = train.at[index, 'Jinete'].replace('. ', '.')
      train.at[index,'Preparador'] = train.at[index,'Preparador'].replace('. ', '.')
      train.at[index,'Propietario'] = train.at[index,'Propietario'].replace('. ', '.')

removePointSS()

print('asd')

In [ ]:
def eraseBlank(list):
  aux = []
  for element in list:
    print(element)
    element = element.strip()
    aux.append(element)
  return aux

test['NombreCaballo'] = test['NombreCaballo'].str.partition('(')[0]
listaNombreCaballo = test['NombreCaballo'].unique().tolist()

listTestHorse= eraseBlank(listaNombreCaballo)

print(listTestHorse)

In [ ]:
listTestHorse.sort()
print(listTestHorse)

In [ ]:
test['Jinete'] = test['Jinete'].str.partition('(')[0]
lista = test['Jinete'].unique().tolist()

listTestJockey = eraseBlank(lista)

test['Preparador'] = test['Preparador'].str.partition('(')[0]
lista = test['Preparador'].unique().tolist()

listTestTrainer = eraseBlank(lista)

test['Propietario'] = test['Propietario'].str.partition('(')[0]
lista = test['Propietario'].unique().tolist()

listTestOwner = eraseBlank(lista)



In [ ]:
train.info()

In [ ]:
def changeNotFoundInListHorse(listTrain, listTest):
  index = 0
  listaApariciones = []
  for element in listTrain:
    if(element not in listTest):
      print("No Encontrado:", element)
      train.at[index, 'NombreCaballo'] = 'OtherHorse'
    else:
      print("Encontrado:", element)
      listaApariciones.append(element)
    index += 1
  return listaApariciones


train['NombreCaballo'] = train['NombreCaballo'].str.partition('(')[0]
listTrainHorse = train['NombreCaballo'].tolist()
listTrainHorse= eraseBlank(listTrainHorse)

## Calculo Contricantes
---

In [ ]:
def calculoContrincantes(listaF, listaH, longitud):
  for fecha in listaF:
    for horario in listaH:
      index = 0
      listaIndex = []
      sum = 0
      while(index < longitud):
        if((train.at[index, 'Fecha'] == fecha ) and (train.at[index, 'Hora'] == horario )):
          listaIndex.append(index)
          sum += 1
        index += 1
      for indice in listaIndex:
        train.at[indice, 'Contrincantes'] = sum


listaFechas = train['Fecha'].unique().tolist()
listaHorarios = train['Hora'].unique().tolist()
longitud = len(train.index)

calculoContrincantes(listaFechas, listaHorarios, longitud)




## Calculo Raiz Puesto
---

In [ ]:
def calculoRaizPuesto(index):
  i = 0
  while(i < index):
    train.at[i, 'Puesto'] = float(((train.at[i, 'Puesto'] / (train.at[i, 'Contrincantes'])**(1. / 3.))))
    # train.at[i, 'PuestoOg'] = float(((train.at[i, 'PuestoOg'] / (train.at[i, 'Contrincantes'])**(1. / 3.))))
    i += 1

train['Puesto'] = train['Puesto'].astype(float)
calculoRaizPuesto(len(train.index))


In [ ]:
pd.set_option('display.max_rows', None)
train[['NombreCaballo','Puesto','Contrincantes']].head(501)

## **Busqueda Caballos Ganadores en ciertas condiciones**
---

### Busqueda UltimasActuacionesCorto
---

In [ ]:
from datetime import timedelta


def ultimasActuacionesIter(indices):
  for i in range(indices):
    print(i)
    puesto = train.at[i, 'PuestoOg'] #Para Train
    dividendo = train.at[i, 'Dividendo']
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    print("Puesto Actual -><", puesto ," y Fecha Actual -->", fecha)

    media = 0.0 # Media carreras
    sum = 0.0 # Numero Carreras

    media_odds = 0.0
    sum_odds = 0.0

    for j in range(indices):
      fechaAux = train.at[j, 'FechaAux']
      nombreAux= train.at[j, 'NombreCaballo']
      sinceDate = fecha - timedelta(days=90)
      recentDates = fecha - timedelta(days = 15)
      # print("Since ->", sinceDate, "/Fecga-->", fechaAux, "/Resul -->", (sinceDate < fechaAux) )
      if((nombreAux == nombre) and (fechaAux < fecha) and (sinceDate < fechaAux)):
        print("Valor Puesto-->", train.at[j, 'PuestoOg'], "/Fecha -->", fechaAux)
        value = 0.0
        value = (train.at[j, 'PuestoOg'] )
        media += value
        sum += 1

        # if(recentDates < fechaAux):
          # value = value**(1.3);
        odds = 0.0
        odds = (train.at[j, 'Dividendo'])
        if((train.at[j, 'Hipodromo'] != 'Sanlucar')):
          media_odds += odds
          sum_odds += 1

    print("Suma Medias -->", media, "/", sum, "<-- Nº Act Anteriores")
    # Puesto
    if(sum != 0):
      media /= sum
    if(sum == 0):
      media = puesto
      train.at[i, 'UltimasActuacionesCorto'] = media
    else:
      train.at[i, 'UltimasActuacionesCorto'] = media
    print(i, "-->", nombre, " MediaUltAct->", media)

    # Dividendo
    if(sum_odds != 0):
      media_odds /= sum_odds
    if(sum_odds == 0):
      media_odds = dividendo
      train.at[i, 'UltimosDividendosCorto'] = media_odds
    else:
      train.at[i, 'UltimosDividendosCorto'] = media_odds

indices = len(train.index)
ultimasActuacionesIter(indices)

In [ ]:
train['UltimasActuacionesCorto'][:20].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda UltimasActuacionesMedio
---

In [ ]:
from datetime import timedelta


def ultimasActuacionesIter(indices):
  for i in range(indices):
    print(i)
    puesto = train.at[i, 'PuestoOg'] #Para Train
    dividendo = train.at[i, 'Dividendo']
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    print("Puesto Actual -><", puesto ," y Fecha Actual -->", fecha)

    media = 0.0 # Media carreras
    sum = 0.0 # Numero Carreras

    media_odds = 0.0
    sum_odds = 0.0

    for j in range(indices):
      fechaAux = train.at[j, 'FechaAux']
      nombreAux= train.at[j, 'NombreCaballo']
      sinceDate = fecha - timedelta(days=120)
      recentDates = fecha - timedelta(days = 22)
      # print("Aince ->", sinceDate, "/Fecga-->", fechaAux, "/Resul -->", (sinceDate < fechaAux) )
      if((nombreAux == nombre) and (fechaAux < fecha) and (sinceDate < fechaAux)):
        print("Valor Puesto-->", train.at[j, 'PuestoOg'], "/Fecha -->", fechaAux)
        value = 0.0
        value = (train.at[j, 'PuestoOg'])
        media += value
        sum += 1
        # if(recentDates < fechaAux):
          # value = value**(1.4);

        odds = 0.0
        odds = (train.at[j, 'Dividendo'])
        if((train.at[j, 'Hipodromo'] != 'Sanlucar')):
          media_odds += odds
          sum_odds += 1

    print("Suma Medias -->", media, "/", sum, "<-- Nº Act Anteriores")
    if(sum != 0):
      media /= sum
    if(sum == 0):
      media = puesto
      train.at[i, 'UltimasActuacionesMedio'] = media
    else:
      train.at[i, 'UltimasActuacionesMedio'] = media
    print(i, "-->", nombre, " MediaUltAct->", media)

    # Dividendo
    if(sum_odds != 0):
      media_odds /= sum_odds
    if(sum_odds == 0):
      media_odds = dividendo
      train.at[i, 'UltimosDividendosMedio'] = media_odds
    else:
      train.at[i, 'UltimosDividendosMedio'] = media_odds



indices = len(train.index)
ultimasActuacionesIter(indices)

In [ ]:
train['UltimasActuacionesMedio'][:20].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda UltimasActuacionesLargo
---

In [ ]:
from datetime import timedelta


def ultimasActuacionesIter(indices):
  for i in range(indices):
    print(i)
    puesto = train.at[i, 'PuestoOg'] #Para Train
    dividendo = train.at[i, 'Dividendo'] #Para Train

    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    print("Puesto Actual -><", puesto ," y Fecha Actual -->", fecha)

    media = 0.0 # Media carreras
    sum = 0.0 # Numero Carreras

    media_odds = 0.0
    sum_odds = 0.0

    for j in range(indices):
      fechaAux = train.at[j, 'FechaAux']
      nombreAux= train.at[j, 'NombreCaballo']
      sinceDate = fecha - timedelta(days=200)
      recentDates = fecha - timedelta(days = 30)
      # print("Aince ->", sinceDate, "/Fecga-->", fechaAux, "/Resul -->", (sinceDate < fechaAux) )
      if((nombreAux == nombre) and (fechaAux < fecha) and (sinceDate < fechaAux)):
        print("Valor Puesto-->", train.at[j, 'PuestoOg'], "/Fecha -->", fechaAux)
        value = 0.0
        value = (train.at[j, 'PuestoOg'] )
        media += value
        sum += 1
        # if(recentDates < fechaAux):
          # value = value**(1.5);

        odds = 0.0
        odds = (train.at[j, 'Dividendo'])
        if((train.at[j, 'Hipodromo'] != 'Sanlucar')):
          media_odds += odds
          sum_odds += 1

    print("Suma Medias -->", media, "/", sum, "<-- Nº Act Anteriores")
    if(sum != 0):
      media /= sum
    if(sum == 0):
      media = puesto
      train.at[i, 'UltimasActuacionesLargo'] = media
    else:
      train.at[i, 'UltimasActuacionesLargo'] = media
    print(i, "-->", nombre, " MediaUltAct->", media)

    # Dividendo
    if(sum_odds != 0):
      media_odds /= sum_odds
    if(sum_odds == 0):
      media_odds = dividendo
      train.at[i, 'UltimosDividendosLargo'] = media_odds
    else:
      train.at[i, 'UltimosDividendosLargo'] = media_odds



indices = len(train.index)
ultimasActuacionesIter(indices)

In [ ]:
train['UltimasActuacionesLargo'][:20].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda UltimasDistancia(Corto/Medio/Largo)
---

In [ ]:
from datetime import timedelta


def ultimasDistanciasCorto():
  for i in range(len(train.index)):
    print(i)
    distancia = train.at[i, 'DistanciaMeta'] #Para Train
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    print("Distancia Actual -><", distancia ," y Fecha Actual -->", fecha)

    media = 0.0 # Media carreras
    sum = 0.0 # Numero Carreras

    for j in range(len(train.index)):
      fechaAux = train.at[j, 'FechaAux']
      nombreAux= train.at[j, 'NombreCaballo']
      sinceDate = fecha - timedelta(days=60)
      # print("Aince ->", sinceDate, "/Fecga-->", fechaAux, "/Resul -->", (sinceDate < fechaAux) )
      if((nombreAux == nombre) and (fechaAux < fecha) and (sinceDate < fechaAux)):
        print("Valor Distancia-->", train.at[j, 'DistanciaMeta'], "/Fecha -->", fechaAux)
        value = 0.0
        value = (train.at[j, 'DistanciaMeta'] )
        media += value
        sum += 1
    print("Suma Medias -->", media, "/", sum, "<-- Nº Act Anteriores")
    if(sum != 0):
      media /= sum
    if(sum == 0):
      media = distancia
      train.at[i, 'ultimasDistanciasCorto'] = media
    else:
      train.at[i, 'ultimasDistanciasCorto'] = media
    print(i, "-->", nombre, " MediaUltAct->", media)



ultimasDistanciasCorto()

In [ ]:
from datetime import timedelta


def ultimasDistanciasMedio():
  for i in range(len(train.index)):
    print(i)
    distancia = train.at[i, 'DistanciaMeta'] #Para Train
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    print("Distancia Actual -><", distancia ," y Fecha Actual -->", fecha)

    media = 0.0 # Media carreras
    sum = 0.0 # Numero Carreras

    for j in range(len(train.index)):
      fechaAux = train.at[j, 'FechaAux']
      nombreAux= train.at[j, 'NombreCaballo']
      sinceDate = fecha - timedelta(days=120)
      # print("Aince ->", sinceDate, "/Fecga-->", fechaAux, "/Resul -->", (sinceDate < fechaAux) )
      if((nombreAux == nombre) and (fechaAux < fecha) and (sinceDate < fechaAux)):
        print("Valor Distancia-->", train.at[j, 'DistanciaMeta'], "/Fecha -->", fechaAux)
        value = 0.0
        value = (train.at[j, 'DistanciaMeta'] )
        media += value
        sum += 1
    print("Suma Medias -->", media, "/", sum, "<-- Nº Act Anteriores")
    if(sum != 0):
      media /= sum
    if(sum == 0):
      media = distancia
      train.at[i, 'ultimasDistanciasMedio'] = media
    else:
      train.at[i, 'ultimasDistanciasMedio'] = media
    print(i, "-->", nombre, " MediaUltAct->", media)



ultimasDistanciasMedio()

In [ ]:
from datetime import timedelta


def ultimasDistanciasLargo():
  for i in range(len(train.index)):
    print(i)
    distancia = train.at[i, 'DistanciaMeta'] #Para Train
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    print("Distancia Actual -><", distancia ," y Fecha Actual -->", fecha)

    media = 0.0 # Media carreras
    sum = 0.0 # Numero Carreras

    for j in range(len(train.index)):
      fechaAux = train.at[j, 'FechaAux']
      nombreAux= train.at[j, 'NombreCaballo']
      sinceDate = fecha - timedelta(days=200)
      # print("Aince ->", sinceDate, "/Fecga-->", fechaAux, "/Resul -->", (sinceDate < fechaAux) )
      if((nombreAux == nombre) and (fechaAux < fecha) and (sinceDate < fechaAux)):
        print("Valor Distancia-->", train.at[j, 'DistanciaMeta'], "/Fecha -->", fechaAux)
        value = 0.0
        value = (train.at[j, 'DistanciaMeta'] )
        media += value
        sum += 1
    print("Suma Medias -->", media, "/", sum, "<-- Nº Act Anteriores")
    if(sum != 0):
      media /= sum
    if(sum == 0):
      media = distancia
      train.at[i, 'ultimasDistanciasLargo'] = media
    else:
      train.at[i, 'ultimasDistanciasLargo'] = media
    print(i, "-->", nombre, " MediaUltAct->", media)



ultimasDistanciasLargo()

### Busqueda Distancia
---

In [ ]:
def comprobacionTipoDistancia(indice):
  tipo = 0   # 0, 1, 2 / Corta, Media, Larga
  val = train.at[indice, 'Distancia']
  if(val >= 3200):
    tipo = 2
  if((val >= 1600) & (val <= 2400)):
    tipo = 1
  return tipo;

def destrezaDistancia(indices):
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoDistancia(i)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0
    sumAux = 0
    for j in range(indices):
      fechaAux = train.at[j, 'FechaAux']
      tipoAux = comprobacionTipoDistancia(j)
      nombreAux= train.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (train.at[j, 'PuestoOg'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (train.at[j, 'PuestoOg'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/") # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaDistancia'] = 1
    else:
      train.at[i, 'DestrezaDistancia'] = 0


indices = len(train.index)
trainOg = train
destrezaDistancia(indices)


#Len Intervalos
print(len(train[(train['Distancia'] < 1600).tolist()]))
print(len(train[((train['Distancia'] >= 1600) & (train['Distancia'] <= 2400) ).tolist()]))
print(len(train[(train['Distancia'] >= 3200).tolist()]))

In [ ]:
train['DestrezaDistancia'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
pd.set_option('display.max_rows', None)
train[['NombreCaballo','DestrezaDistancia']].head(501)

### Busqueda Meteorología
---

In [ ]:
train['Meteorología'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoMeteorologia(indice, listaMeteo):
  tipo = -1 # 0 -> Lluvia / 1 -> Nublado / 2 -> Soleado, Despejado
  val = train.at[indice, 'Meteorología']
  for meteo in listaMeteo:
   if("Lluvia" in val or "precipitaciones" in val):
      tipo = 1
   elif("Soleado" in val or "Despejado" in val):
      tipo = 2
   else:
     tipo = 0
  return tipo;

def destrezaMeteorologia(indices, listaMeteo):
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoMeteorologia(i, listaMeteo)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0
    sumAux = 0
    for j in range(indices):
      fechaAux = train.at[j, 'FechaAux']
      tipoAux = comprobacionTipoMeteorologia(j, listaMeteo)
      nombreAux= train.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (train.at[j, 'PuestoOg'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (train.at[j, 'PuestoOg'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/")
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaMeteo'] = 1
    else:
      train.at[i, 'DestrezaMeteo'] = 0


indices = len(train.index)
listaMeteo = train['Meteorología'].unique().tolist()
destrezaMeteorologia(indices, listaMeteo)

In [ ]:
train['DestrezaMeteo'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda Lluvia
---

In [ ]:
train['LLuvia'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoLLuvia(indice, listaLLuvia):
  tipo = -1 # 0 -> 0.0 / 1 -> > 0.0 / 2 -> > 1
  val = train.at[indice, 'LLuvia']
  for meteo in listaMeteo:
   if(val > 0):
      tipo = 1
   elif(val > 1):
      tipo = 2
   else:
     tipo = 0
  return tipo;

def destrezaMeteorologia(indices, listaLLuvia):
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoLLuvia(i, listaLLuvia)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0
    sumAux = 0
    for j in range(indices):
      fechaAux = train.at[j, 'FechaAux']
      tipoAux = comprobacionTipoLLuvia(j, listaLLuvia)
      nombreAux= train.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (train.at[j, 'PuestoOg'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (train.at[j, 'PuestoOg'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/")
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaLLuvia'] = 1
    else:
      train.at[i, 'DestrezaLLuvia'] = 0


indices = len(train.index)
listaLLuvia = train['LLuvia'].unique().tolist()
destrezaMeteorologia(indices, listaLLuvia)

In [ ]:
train['DestrezaLLuvia'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda Temperatura
---

In [ ]:
train['Temperatura'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoTemperatura(indice, listaTemperatura):
  tipo = -1 # 0 -> < 20 / 1 -> > 30 > x < 20 / 2 -> > > 30
  val = train.at[indice, 'Temperatura']
  for meteo in listaMeteo:
   if(val > 30):
      tipo = 2
   elif(val < 20):
      tipo = 0
   else:
     tipo = 0
  return tipo;

def destrezaTemperatura(indices, listaTemperatura):
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoTemperatura(i, listaTemperatura)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0
    sumAux = 0
    for j in range(indices):
      fechaAux = train.at[j, 'FechaAux']
      tipoAux = comprobacionTipoTemperatura(j, listaTemperatura)
      nombreAux= train.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (train.at[j, 'PuestoOg'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (train.at[j, 'PuestoOg'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/")
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaTemperatura'] = 1
    else:
      train.at[i, 'DestrezaTemperatura'] = 0


indices = len(train.index)
listaTemperatura = train['Temperatura'].unique().tolist()
destrezaTemperatura(indices, listaTemperatura)

In [ ]:
train['DestrezaTemperatura'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda Viento
---

In [ ]:
train['Viento'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoViento(indice, listaViento):
  tipo = -1 # 0 -> < 5 / 1 -> > 30 > x > 5 / 2 -> > > 30
  val = train.at[indice, 'Viento']
  for meteo in listaMeteo:
   if(val > 30):
      tipo = 2
   elif(val < 5):
      tipo = 0
   else:
     tipo = 0
  return tipo;

def destrezaViento(indices, listaViento):
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoViento(i, listaViento)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0
    sumAux = 0
    for j in range(indices):
      fechaAux = train.at[j, 'FechaAux']
      tipoAux = comprobacionTipoViento(j, listaViento)
      nombreAux= train.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (train.at[j, 'PuestoOg'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (train.at[j, 'PuestoOg'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/")
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaViento'] = 1
    else:
      train.at[i, 'DestrezaViento'] = 0


indices = len(train.index)
listaViento = train['Viento'].unique().tolist()
destrezaViento(indices, listaViento)

In [ ]:
train['DestrezaViento'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda Terreno
---

In [ ]:
train['Terreno'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoTerreno(indice, listaTerreno):
  tipo = -1 # 0 -> Arena / 1 -> H - Blando / 2 -> H - Bueno
  val = train.at[indice, 'Terreno']
  if(val == "H-Bueno"):
    val = val.replace('H-Bueno', 'H - Bueno')
    train.at[indice, 'Terreno'] = val
  for meteo in listaMeteo:
   if(val == 'H - Bueno'):
      tipo = 2
   elif(val == 'H - Blando'):
      tipo = 1
   elif(val == 'Arena'):
      tipo = 0
   elif(val == 'Playa'):
      tipo = 3
   else:
      tipo = 4
  return tipo;

def destrezaTerreno(indices, listaTerreno):
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoTerreno(i, listaTerreno)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0
    sumAux = 0
    for j in range(indices):
      fechaAux = train.at[j, 'FechaAux']
      tipoAux = comprobacionTipoTerreno(j, listaTerreno)
      nombreAux= train.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (train.at[j, 'PuestoOg'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (train.at[j, 'PuestoOg'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/")
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaTerreno'] = 1
    else:
      train.at[i, 'DestrezaTerreno'] = 0


indices = len(train.index)
listaTerreno = train['Terreno'].unique().tolist()
destrezaTerreno(indices, listaTerreno)

In [ ]:
train['DestrezaTerreno'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Peso)**

In [ ]:
train['Peso'] = train['Peso'].str.partition('-')[0]
train['Peso'] = train['Peso'].str[:2]



In [ ]:
train['Peso'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Edad)**

In [ ]:
train['Edad'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Mantilla)**

In [ ]:
train['Mantilla'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Propietario)(Train)**

In [ ]:
def changeNotFoundInListOwner(listTrain, listTest):
  index = 0
  listaApariciones = []
  for element in listTrain:
    if(element not in listTest):
      print("No Encontrado:", element)
      train.at[index, 'Propietario'] = 'OtherOwner'
    else:
      print("Encontrado:", element)
      listaApariciones.append(element)
    index += 1
  return listaApariciones



#Lógica de lista de Test en "Tratamiento(NombreCaballo)"

listTrainOwner = train['Propietario'].tolist()
listTrainOwner= eraseBlank(listTrainOwner)


## **Tratamiento (Preparador)(Train)**

In [ ]:
def changeNotFoundInListTrainer(listTrain, listTest):
  index = 0
  listaApariciones = []
  for element in listTrain:
    if(element not in listTest):
      print("No Encontrado:", element)
      train.at[index, 'Preparador'] = 'OtherTrainer'
    else:
      print("Encontrado:", element)
      listaApariciones.append(element)
    index += 1
  return listaApariciones

#Lógica de lista de Test en "Tratamiento(NombreCaballo)"

listTrainTrainer = train['Preparador'].tolist()
listTrainTrainer= eraseBlank(listTrainTrainer)


## **Tratamiento (Jinete)(Train)**

In [ ]:
def changeNotFoundInListJockey(listTrain, listTest):
  index = 0
  listaApariciones = []
  for element in listTrain:
    if(element not in listTest):
      print("No Encontrado:", element)
      train.at[index, 'Jinete'] = 'OtherJockey'
    else:
      print("Encontrado:", element)
      listaApariciones.append(element)
    index += 1
  return listaApariciones

#Lógica de lista de Test en "Tratamiento(NombreCaballo)"

train['Jinete'] = train['Jinete'].str.partition('(')[0]
listTrainJockey = train['Jinete'].tolist()
listTrainJockey= eraseBlank(listTrainJockey)

print('asd')


## **Tratamiento Change Not Found In Test(NombreCaballo)(Propieatario)(Preparador)(Jinete)**

In [ ]:
listTestHorse = [x.upper() for x in listTestHorse]
listTestOwner = [x.upper() for x in listTestOwner]
listTestTrainer = [x.upper() for x in listTestTrainer]
listTestJockey = [x.upper() for x in listTestJockey]



In [ ]:
listAparicionesHorse= changeNotFoundInListHorse(listTrainHorse, listTestHorse)

train['Propietario'] = train['Propietario'].str.strip()
train['NombreCaballo'] = train['NombreCaballo'].str.strip()
train['Jinete'] = train['Jinete'].str.strip()
train['Preparador'] = train['Preparador'].str.strip()



In [ ]:
train['NombreCaballo'].head(45)

In [ ]:
train['Propietario'] = train['Propietario'].str.partition('(')[0]

listAparicionesOwner = changeNotFoundInListOwner(listTrainOwner, listTestOwner)
train['Propietario'].head(15)

In [ ]:
listAparicionesTrainer = changeNotFoundInListTrainer(listTrainTrainer, listTestTrainer)
train['Preparador'].head(15)

In [ ]:
print(listTestJockey)

In [ ]:
listAparicionesJockey = changeNotFoundInListJockey(listTrainJockey, listTestJockey)
train['Jinete'].head(15)

In [ ]:
train['NombreCaballo'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Problemas)**


In [ ]:
train['Problemas'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
train['Problemas'] = train['Problemas'].fillna('0')
train['Problema_Nulo'] = np.where(train['Problemas'].str.contains('0'),1,0)
train['Problema_1'] = np.where(train['Problemas'].str.contains('1'),1,0)
train['Problema_2'] = np.where(train['Problemas'].str.contains('2'),1,0)
train['Problema_3'] = np.where(train['Problemas'].str.contains('3'),1,0)
train['Problema_4'] = np.where(train['Problemas'].str.contains('4'),1,0)
train['Problema_5'] = np.where(train['Problemas'].str.contains('5'),1,0)
train['Problema_6'] = np.where(train['Problemas'].str.contains('6'),1,0)
train['Problema_7'] = np.where(train['Problemas'].str.contains('7'),1,0)
train['Problema_8'] = np.where(train['Problemas'].str.contains('8'),1,0)
train.head()



## **Tratamiento (UltimasActuaciones)**


In [ ]:
# def count_lastRaces(arrayLast):
#   index = 0
#   for element in arrayLast:
#     element = element.replace(',', '')
#     element = element.replace('[', '')
#     element = element[:len(element)-1]
#     i = 0
#     j = 0
#     sum = 0
#     if 'Debutante' in element:
#       sum = 10
#       element = []
#     else:
#       element = element.replace('NP', '08')
#       element = element.replace('Desc', '08')
#       element = element.replace(' ', '')
#       while(i < len(element)):
#         if(i == 0 and len(element)>4):
#           j = j+2
#           sum += int(element[i:j])**1.3
#           i = i+2
#         elif (i == 2 and len(element)>4):
#           j = j+2
#           sum += int(element[i:j])**1.2
#           i = i+2
#         else:
#           j = j+2
#           print(index)
#           sum += int(element[i:j])
#           i = i+2
#       print('******INDEX ', index+2, '*****')
#       print('Sum -->', sum, '/Len -->', len(element))
#       sum = sum / len(element)
#       print('Result -->', sum, '/Element -->' ,element)
#     train.at[index, 'MediaUltimasActuaciones'] = sum
#     train.at[index, 'CantidadActuaciones'] = (len(element)/2)
#     index += 1





# t_array = train["UltimasActuaciones"]
# count_lastRaces(t_array)

In [ ]:
train[['NombreCaballo','UltimasActuacionesLargo']].head(128)

## **Tratamiento (Hora)**


In [ ]:
train['Hora'] = train['Hora'].str.replace(':', '')

## **Tratamiento (Terreno)**


In [ ]:
train.info()

## **Tratamiento (Distancia)**


## **Tratamiento (Tipo)**


## **Tratamiento (SentidoHipodromo)**


## **Tratamiento (Meteorologia)**


## **Tratamiento (Lluvia)**


## **Tratamiento (Viento)**


## **Tratamiento (Hipodromo)**


# Visualizacion

In [ ]:
train.describe()

In [ ]:
train.hist(figsize = (40,30))

In [ ]:
# train['Puesto'].corr(train['MediaUltimasActuaciones'])

In [ ]:
from scipy.stats import kendalltau, pearsonr, spearmanr
def kendall_pval(x,y):
    return kendalltau(x,y)[1]

def pearsonr_pval(x,y):
    return pearsonr(x,y)[1]

def spearmanr_pval(x,y):
    return spearmanr(x,y)[1]


print("Correlation -->", train['Puesto'].corr(train['UltimasActuacionesLargo']))
print(train['Puesto'].corr(train['UltimasActuacionesLargo'], method=kendall_pval))
print(train['Puesto'].corr(train['UltimasActuacionesLargo'], method=pearsonr_pval))
print(train['Puesto'].corr(train['UltimasActuacionesLargo'], method=spearmanr_pval))

print("Correlation -->", train['Puesto'].corr(train['UltimasActuacionesCorto']))
print(train['Puesto'].corr(train['UltimasActuacionesCorto'], method=kendall_pval))
print(train['Puesto'].corr(train['UltimasActuacionesCorto'], method=pearsonr_pval))
print(train['Puesto'].corr(train['UltimasActuacionesCorto'], method=spearmanr_pval))

In [ ]:
C_mat = train.corr()
fig = plt.figure(figsize = (15,15))

#Correlation ranges from -1 to +1. Values closer to zero means there is no linear trend between the two variables.
#The close to 1 the correlation is the more positively correlated they are; that is as one increases so does the other and the closer to 1 the stronger this relationship is.
#A correlation closer to -1 is similar, but instead of both increasing one variable will decrease as the other increases.

sns.heatmap(C_mat, vmax = .8, square = True, cmap="PiYG")
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(data=train, x='Distancia', y='Puesto')

plt.show()

In [ ]:
print(len(train[(train['Distancia'] < 1600).tolist()]))

print(len(train[((train['Distancia'] >= 1600) & (train['Distancia'] <= 2400) ).tolist()]))

print(len(train[(train['Distancia'] >= 3200).tolist()]))

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(data=train, x='Hora', y='Puesto')

plt.show()

In [ ]:
plt.figure(figsize=(50,8))
sns.boxplot(data=train, x='Tipo', y='Puesto')

plt.show()

In [ ]:
plt.figure(figsize=(40,8))
sns.boxplot(data=train, x='Meteorología', y='Puesto')

plt.show()

In [ ]:
plt.figure(figsize=(40,8))
sns.boxplot(data=train, x='LLuvia', y='Puesto')

plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(data=train, x='Viento', y='Puesto')

plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(data=train, x='Temperatura', y='Puesto')

plt.show()

# Preprocesamiento Final

In [ ]:
train.info()

In [ ]:
pd.set_option('display.max_columns', None)
train.head(20)

In [ ]:
# def conversion(): #Colocado
#   for i in range(len(train.index)):
#     if(train.at[i ,'Contrincantes'] < 11):
#       if(train.at[i ,'PuestoOg'] == 1 or train.at[i, 'PuestoOg'] == 2):
#         train.at[i, 'Puesto'] = 1
#       else:
#         train.at[i, 'Puesto'] = 0
#     else:
#       if(train.at[i, 'PuestoOg'] == 1 or train.at[i, 'PuestoOg'] == 2 or train.at[i, 'PuestoOg'] == 3):
#          train.at[i, 'Puesto'] = 1
#       else:
#         train.at[i, 'Puesto'] = 0

# conversion()

In [ ]:
def conversion(): #Ganadores
    for i in range(len(train.index)):
      if(train.at[i ,'PuestoOg'] == 1):
        train.at[i, 'Puesto'] = 1
      else:
        train.at[i, 'Puesto'] = 0

conversion()

In [ ]:
# train['PuestoChanged'] = train['Puesto'] //Temporal
trainPuesto = train

In [ ]:
print(train['DistanciaMeta'].unique().tolist())


In [ ]:
import sklearn
import sklearn.compose
#Solo ejecutar previo a esto product para separar las palabras
trainDistancia = train
column_transformer_train = sklearn.compose.ColumnTransformer(transformers=[
    ("drop", "drop", ['Fecha','year','day', 'Problemas','Hora', 'FechaAux', 'Otoño', 'month', 'Distancia', 'Mantilla', 'Terreno', 'Tipo', 'Meteorología', 'LLuvia', 'Viento', 'Temperatura', 'Hipodromo', 'Contrincantes', 'DistanciaMeta', 'PuestoOg']),
    ("scale", sklearn.preprocessing.StandardScaler(), []),
    # ("num", sklearn.preprocessing.MinMaxScaler(), ['Distancia', 'Edad', 'Peso']),
    ("one-hot", sklearn.preprocessing.OneHotEncoder(handle_unknown="ignore"), ['NombreCaballo','Propietario', 'Preparador', 'Jinete']),
], remainder='passthrough');

# column_transformer = sklearn.compose.ColumnTransformer(transformers=[
#     ("drop", "drop", ['UltimasActuaciones','Fecha','year','day', 'Problemas','Hora']),
#     ("scale", sklearn.preprocessing.StandardScaler(), []),
#     ("num", sklearn.preprocessing.MinMaxScaler(), ['Distancia', 'Edad', 'Peso']),
#     ("one-hot", sklearn.preprocessing.OneHotEncoder(handle_unknown="ignore"), ['NombreCaballo', 'Mantilla','Propietario', 'Preparador', 'Jinete', 'FechaAux', 'Terreno', 'Tipo', 'Meteorología', 'LLuvia', 'Viento', 'TemperaturaMax', 'TemperaturaMin', 'Hipodromo']),
# ], remainder='passthrough');

In [ ]:
train = train.reset_index()
X=train
X = X.drop(["Puesto"], axis = 1)
X_trainAuxNN = X

In [ ]:
X_transform = column_transformer_train.fit_transform(X)

X_transformAux = pd.DataFrame(column_transformer_train.fit_transform(X).toarray())

In [ ]:

# from scipy.sparse import csr_matrix

# # create a NumPy array

# X_transform_matrix = csr_matrix(X_transform)
# X_transformAux = pd.DataFrame(X_transform_matrix.toarray())

In [ ]:
column_transformer_train.get_feature_names_out()

In [ ]:
train
X_transform
X = X_transform
y = train["Puesto"] #Nos quedamos con todo menos la columna objetivo, en este caso 'price'


X_trainNN = X
X_trainArrayNN = X_transformAux

y_trainNN = y

In [ ]:
# train[['NombreCaballo','MediaUltimasActuaciones']].head(128)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1920)

# Optimización
---

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor

#Regresión lineal
linear = LinearRegression()
linear.fit(X_train, y_train)

#Regresión de Ridge

ridge = Ridge()
ridge.fit(X_train, y_train)

#LASSO

lasso = linear_model.Lasso()
lasso.fit(X_train, y_train)

#ElasticNet

elasticnet = ElasticNet()
elasticnet.fit(X_train, y_train);


reg = SGDRegressor(max_iter=50, random_state=43)
reg.fit(X_train, y_train);

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

for i in range(0, 11):
  for j in range(0, 11):
    #     xgb_model = xgb.XGBRegressor(objective = 'reg:squarederror', reg_alpha=i*0.1, reg_lambda=j*0.1, verbosity=0, random_state=42, eta=0.2, tree_method="exact") 148

    elasticnet = ElasticNet(alpha=i*0.1, l1_ratio=j*0.1)
    elasticnet.fit(X_train, y_train)
    print('ElasticNet (train): ' + str(mean_absolute_error(y_train, elasticnet.predict(X_train))))
    print(f'{i} {j} ' + str(mean_absolute_error(y_test, elasticnet.predict(X_test))))

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

for i in range(0, 11):
  for j in range(0, 11):
    #     xgb_model = xgb.XGBRegressor(objective = 'reg:squarederror', reg_alpha=i*0.1, reg_lambda=j*0.1, verbosity=0, random_state=42, eta=0.2, tree_method="exact") 148

    xgb_model = xgb.XGBRegressor(objective = 'reg:squarederror', reg_alpha=i*0.1, reg_lambda=j*0.1, verbosity=0, random_state=42, eta=0.2, tree_method="exact") #subsample ojo
    xgb_model.fit(X_train, y_train)
    print('ElasticNet (train): ' + str(mean_absolute_error(y_train, xgb_model.predict(X_train))))
    print(f'{i} {j} ' + str(mean_absolute_error(y_test, xgb_model.predict(X_test))))

In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(objective = 'reg:squarederror', reg_alpha=7*0.1, reg_lambda=4*0.1, verbosity=0, random_state=42, eta=0.2, tree_method="exact")

xgb_model.fit(X, y)
#print('Lineal (train): ' + str(mean_absolute_error(y_train, xgb_model.predict(X))))
#print('Lineal (test): ' + str(mean_absolute_error(y_test, xgb_model.predict(X_test))))

In [ ]:
!pip install catboost
import catboost as cb

In [ ]:
params = {"depth": 10, "learning_rate": 0.5, "iterations": 1, "l2_leaf_reg": 5} #Iterations CatBoost
model_cat_tun = cb.CatBoostClassifier(**params)
model_cat_tun.fit(X,y)

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor


# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 2000, random_state = 42)
# Train the model on training data
rf.fit(X, y);

In [ ]:

from sklearn.tree import DecisionTreeRegressor

# Build decision tree
tree=DecisionTreeRegressor(max_depth=20)
tree.fit(X, y);

In [ ]:
from sklearn.metrics import mean_absolute_error


print('RandomForest (train): ' + str(mean_absolute_error(y_train,  rf.predict(X_train))))
print('RandomForest (test): ' + str(mean_absolute_error(y_test,  rf.predict(X_test))))


print('---')


print('XGB (train): ' + str(mean_absolute_error(y_train, xgb_model.predict(X_train))))
print('XGB (test): ' + str(mean_absolute_error(y_test, xgb_model.predict(X_test))))



print('---')

print('Cat (train): ' + str(mean_absolute_error(y_train, model_cat_tun.predict(X_train))))
print('Cat (test): ' + str(mean_absolute_error(y_test, model_cat_tun.predict(X_test))))



print('---')

print('Lineal (train): ' + str(mean_absolute_error(y_train, linear.predict(X_train))))
print('Lineal (test): ' + str(mean_absolute_error(y_test, linear.predict(X_test))))

print('---')

print('Ridge (train): ' + str(mean_absolute_error(y_train, ridge.predict(X_train))))
print('Ridge (test): ' + str(mean_absolute_error(y_test, ridge.predict(X_test))))

print('---')

print('LASSO (train): ' + str(mean_absolute_error(y_train, lasso.predict(X_train))))
print('LASSO (test): ' + str(mean_absolute_error(y_test, lasso.predict(X_test))))

print('---')

print('ElasticNet (train): ' + str(mean_absolute_error(y_train, elasticnet.predict(X_train))))
print('ElasticNet (test): ' + str(mean_absolute_error(y_test, elasticnet.predict(X_test))))

print('---')

print('SGD (train): ' + str(mean_absolute_error(y_train, reg.predict(X_train))))
print('SGD (test): ' + str(mean_absolute_error(y_test, reg.predict(X_test))))

In [ ]:
from sklearn.metrics import accuracy_score

print('Lineal (train): ' + str(linear.score(X_train, y_train)))
print('Lineal (test): ' + str(linear.score(X_test, y_test)))
print('---')
print('Ridge (train): ' + str(ridge.score(X_train, y_train)))
print('Ridge (test): ' + str(ridge.score(X_test, y_test)))
print('---')
print('LASSO (train): ' + str(lasso.score(X_train, y_train)))
print('LASSO (test): ' + str(lasso.score(X_test, y_test)))
print('---')
print('ElasticNet (train): ' + str(elasticnet.score(X_train, y_train)))
print('ElasticNet (test): ' + str(elasticnet.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
print('Lineal (train): ' + str(sqrt(mean_squared_error(y_train, linear.predict(X_train)))))
print('Lineal (test): ' + str(sqrt(mean_squared_error(y_test, linear.predict(X_test)))))
print('---')
print('Ridge (train): ' + str(sqrt(mean_squared_error(y_train, ridge.predict(X_train)))))
print('Ridge (test): ' + str(sqrt(mean_squared_error(y_test, ridge.predict(X_test)))))
print('---')
print('LASSO (train): ' + str(sqrt(mean_squared_error(y_train, lasso.predict(X_train)))))
print('LASSO (test): ' + str(sqrt(mean_squared_error(y_test, lasso.predict(X_test)))))
print('---')
print('ElasticNet (train): ' + str(sqrt(mean_squared_error(y_train, elasticnet.predict(X_train)))))
print('ElasticNet (test): ' + str(sqrt(mean_squared_error(y_test, elasticnet.predict(X_test)))))

Linear
---

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
fit_intercept = [True, False]
copy_x = [True, False]
normalize = [True, False]

In [ ]:
linear_Grid = GridSearchCV(
  estimator = LinearRegression(),
  param_grid = dict(fit_intercept = fit_intercept, copy_X = copy_x),
  cv = 5
)
linear_Grid.fit(X, y)

In [ ]:
linear_Grid.best_estimator_

In [ ]:
linear_Grid.best_params_

In [ ]:
linear_Grid.best_score_

In [ ]:
linear = LinearRegression(fit_intercept=True, copy_X=True)
linear.fit(X_train, y_train)

In [ ]:
print("MAE: ")
print('Train: ' + str(mean_absolute_error(y_train, linear.predict(X_train))))
print('Test: ' + str(mean_absolute_error(y_test, linear.predict(X_test))))
print("---------------------------------------")
print("R^2: ")
print("Train: " + str(linear.score(X_train, y_train)))
print("Test: " + str(linear.score(X_test, y_test)))
print("---------------------------------------")
print("RMST: ")
print('Train: ' + str(sqrt(mean_squared_error(y_train, linear.predict(X_train)))))
print('Test: ' + str(sqrt(mean_squared_error(y_train, linear.predict(X_train)))))

Se puede observar que no mejora ya que los parámetros que nos da el GridSearch son los mismos que tiene de base.

Ridge
---

In [ ]:
ridge_Grid = GridSearchCV(
  estimator = Ridge(),
  param_grid = dict(alpha= np.linspace(0, 1, 50), copy_X = copy_x, fit_intercept=fit_intercept),
  cv = 5

)
ridge_Grid.fit(X, y)

In [ ]:
ridge_Grid.best_estimator_

In [ ]:
ridge_Grid.best_params_

In [ ]:
ridge_Grid.best_score_

In [ ]:
ridge = Ridge(alpha=0.32653061224489793, copy_X=True, fit_intercept=True,
       random_state=1337, solver='auto', tol=0.001)
ridge.fit(X_train, y_train)

In [ ]:
print("MAE: ")
print('Train: ' + str(mean_absolute_error(y_train, ridge.predict(X_train))))
print('Test: ' + str(mean_absolute_error(y_test, ridge.predict(X_test))))
print("---------------------------------------")
print("R^2: ")
print("Train: " + str(ridge.score(X_train, y_train)))
print("Test: " + str(ridge.score(X_test, y_test)))
print("---------------------------------------")
print("RMST: ")
print('Train: ' + str(sqrt(mean_squared_error(y_train, ridge.predict(X_train)))))
print('Test: ' + str(sqrt(mean_squared_error(y_test, ridge.predict(X_test)))))

Parece que empeora ligeramente con respecto a los valores por defecto que tenía antes.

LASSO
---

In [ ]:
lasso_Grid = GridSearchCV(estimator = linear_model.Lasso(), param_grid = dict(alpha= np.linspace(0.05,1,20)), cv = 5)
lasso_Grid.fit(X, y)

In [ ]:
lasso_Grid.best_params_

In [ ]:
lasso = linear_model.Lasso(alpha=1, random_state=1337)
lasso.fit(X_train, y_train)

In [ ]:
print("MAE: ")
print('Train: ' + str(mean_absolute_error(y_train, lasso.predict(X_train))))
print('Test: ' + str(mean_absolute_error(y_test, lasso.predict(X_test))))
print("---------------------------------------")
print("R^2: ")
print("Train: " + str(lasso.score(X_train, y_train)))
print("Test: " + str(lasso.score(X_test, y_test)))
print("---------------------------------------")
print("RMST: ")
print('Train: ' + str(sqrt(mean_squared_error(y_train, lasso.predict(X_train)))))
print('Test: ' + str(sqrt(mean_squared_error(y_train, lasso.predict(X_train)))))

Se queda igual que con los valores de base.

ElasticNet
---

Este regresor a veces funciona y a veces explota, dejamos guardado los resultados de cuando ha funcionado. De todos modos nos sale el siguiente error:<br>


```
# /usr/local/lib/python3.6/dist-packages/sklearn linear_model_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge.
```



In [ ]:
elastic_Grid = GridSearchCV(
  estimator = ElasticNet(),
  param_grid = dict(alpha=np.linspace(0.1,1,2), l1_ratio=np.linspace(0,1,2)),
  cv = 5
)
elastic_Grid.fit(X, y)

In [ ]:
elastic_Grid.best_params_

In [ ]:
elasticnet = ElasticNet(alpha = 1.0, l1_ratio = 1, fit_intercept=True, copy_X=True)
elasticnet.fit(X_train, y_train)

In [ ]:
print("MAE: ")
print('Train: ' + str(mean_absolute_error(y_train, elasticnet.predict(X_train))))
print('Test: ' + str(mean_absolute_error(y_test, elasticnet.predict(X_test))))
print("---------------------------------------")
print("R^2: ")
print("Train: " + str(lasso.score(X_train, y_train)))
print("Test: " + str(lasso.score(X_test, y_test)))
print("---------------------------------------")
print("RMST: ")
print('Train: ' + str(sqrt(mean_squared_error(y_train, elasticnet.predict(X_train)))))
print('Test: ' + str(sqrt(mean_squared_error(y_train, elasticnet.predict(X_train)))))

# Validación Cruzada
---

## Linear

MAE

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_validate

In [ ]:
cv_scores_linearMAE = cross_val_score(
                estimator = linear,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_absolute_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_linearMAE}")
print(f"Média métricas de validación cruzada: {cv_scores_linearMAE.mean()}")

R^2

In [ ]:
cv_scores_linearR = cross_val_score(
                estimator = linear,
                X         = X,
                y         = y,
                scoring   = 'r2',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_linearR}")
print(f"Média métricas de validación cruzada: {cv_scores_linearR.mean()}")

RMSE

In [ ]:
cv_scores_linearRMSE = cross_val_score(
                estimator = linear,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_squared_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_linearRMSE}")
print(f"Média métricas de validación cruzada: {cv_scores_linearRMSE.mean()}")

## Ridge
---

MAE

In [ ]:
cv_scores_ridgeMAE = cross_val_score(
                estimator = ridge,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_absolute_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_ridgeMAE}")
print(f"Média métricas de validación cruzada: {cv_scores_ridgeMAE.mean()}")

R^2

In [ ]:
cv_scores_ridgeR = cross_val_score(
                estimator = ridge,
                X         = X,
                y         = y,
                scoring   = 'r2',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_ridgeR}")
print(f"Média métricas de validación cruzada: {cv_scores_ridgeR.mean()}")

RMSE

In [ ]:
cv_scores_ridgeRMSE = cross_val_score(
                estimator = ridge,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_squared_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_ridgeRMSE}")
print(f"Média métricas de validación cruzada: {cv_scores_ridgeRMSE.mean()}")

## LASSO
---

MAE

In [ ]:
cv_scores_lassoMAE = cross_val_score(
                estimator = lasso,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_absolute_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_lassoMAE}")
print(f"Média métricas de validación cruzada: {cv_scores_lassoMAE.mean()}")

R^2

In [ ]:
cv_scores_lassoR = cross_val_score(
                estimator = lasso,
                X         = X,
                y         = y,
                scoring   = 'r2',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_lassoR}")
print(f"Média métricas de validación cruzada: {cv_scores_lassoR.mean()}")

RMSE

In [ ]:
cv_scores_lassoRMSE = cross_val_score(
                estimator = lasso,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_squared_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_lassoRMSE}")
print(f"Média métricas de validación cruzada: {cv_scores_lassoRMSE.mean()}")

## ElasticNet
---

MAE

In [ ]:
cv_scores_netMAE = cross_val_score(
                estimator = elasticnet,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_absolute_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_netMAE}")
print(f"Média métricas de validación cruzada: {cv_scores_netMAE.mean()}")

R^2

In [ ]:
cv_scores_netR = cross_val_score(
                estimator = elasticnet,
                X         = X,
                y         = y,
                scoring   = 'r2',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_netR}")
print(f"Média métricas de validación cruzada: {cv_scores_netR.mean()}")

RMSE

In [ ]:
cv_scores_netRMSE = cross_val_score(
                estimator = elasticnet,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_squared_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_netRMSE}")
print(f"Média métricas de validación cruzada: {cv_scores_netRMSE.mean()}")

## XGBoost
---

MAE

In [ ]:
cv_scores_netMAE = cross_val_score(
                estimator = xgb_model,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_absolute_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_netMAE}")
print(f"Média métricas de validación cruzada: {cv_scores_netMAE.mean()}")

R^2

In [ ]:
cv_scores_netR = cross_val_score(
                estimator = xgb_model,
                X         = X,
                y         = y,
                scoring   = 'r2',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_netR}")
print(f"Média métricas de validación cruzada: {cv_scores_netR.mean()}")

RMSE

In [ ]:
cv_scores_netRMSE = cross_val_score(
                estimator = xgb_model,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_squared_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_netRMSE}")
print(f"Média métricas de validación cruzada: {cv_scores_netRMSE.mean()}")

# Prueba
---

In [ ]:
train = test
train['NombreCaballo'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

# Ajuste Preprocesamiento
- Borrar Puesto
- Ajuste Listas Apariciones

In [ ]:
# train = train.drop('Dividendo', axis=1)

def dividendToFloat():
  for index, row in train.iterrows():
    train.at[index, 'Dividendo'] = str(train.at[index, 'Dividendo']).replace('€', '')
    train.at[index, 'Dividendo'] = str(train.at[index, 'Dividendo']).replace(',', '.')
    if( '-' in str(train.at[index, 'Dividendo'])):
      train.at[index, 'Dividendo'] = '0.0'

dividendToFloat()
train['Dividendo'] = train['Dividendo'].astype(float)

In [ ]:
train['NombreCaballo'] = train['NombreCaballo'].str.partition('(')[0]
print(listAparicionesHorse)
listTestHorse= eraseBlank(listaNombreCaballo)



train['Propietario'] = train['Propietario'].str.strip()
train['NombreCaballo'] = train['NombreCaballo'].str.strip()
train['Jinete'] = train['Jinete'].str.strip()
train['Preparador'] = train['Preparador'].str.strip()

train.loc[~train["Propietario"].isin(listAparicionesOwner), "Propietario"] = "OtherOwner"
# train.loc[~train["NombreCaballo"].isin(listAparicionesHorse), "NombreCaballo"] = "OtherHorse" #Lo hacemos mas tarde
train.loc[~train["Jinete"].isin(listAparicionesJockey), "Jinete"] = "OtherJockey"
train.loc[~train["Preparador"].isin(listAparicionesTrainer), "Preparador"] = "OtherTrainer"

# if 'OtherOwner' not in train:
#   train['one-hot__Propietario_OtherOwner'] = 0

# if 'OtherHorse' not in train:
#   train[ 'one-hot__NombreCaballo_OtherHorse'] = 0

# if 'OtherJockey' not in train:
#   train['one-hot__Jinete_OtherJockey'] = 0

if 'OtherTrainer' not in train:
  train['one-hot__Preparador_OtherTrainer'] = 0


# Preprocesamiento
---

In [ ]:
train['PuestoOg'] = train['Puesto']

In [ ]:
train['Jinete'] = train['Jinete'].str.partition('(')[0]
train['Preparador'] = train['Preparador'].str.partition('(')[0]
train['Propietario'] = train['Propietario'].str.partition('(')[0]
train['NombreCaballo'] = train['NombreCaballo'].str.partition('(')[0]

train.head(5)

## **Tratamiento (DistanciaMeta)**

## **Convertir DistanciaMeta (Comentado porque no existe en Test)**

In [ ]:
# train["DistanciaMeta"] = train["DistanciaMeta"].fillna(0)

In [ ]:
# def conversionDistanciaMetaTrain():
#   train['DistanciaMeta'] = train['DistanciaMeta'].astype('str')
#   for i in range(len(train.index)):
#     decimal = 0
#     if('LEJOS' in train.at[i, 'DistanciaMeta'] or 'Lejos' in train.at[i, 'DistanciaMeta'] or 'lejos' in train.at[i, 'DistanciaMeta']):
#       train.at[i, 'DistanciaMeta'] = '20'
#     if('DISTANCIADO' in train.at[i, 'DistanciaMeta'] or'Distanciado' in train.at[i, 'DistanciaMeta'] or 'distanciado' in train.at[i, 'DistanciaMeta']):
#       train.at[i, 'DistanciaMeta'] = '20'
#     if('CUELLO' in train.at[i, 'DistanciaMeta'] or 'Cuello' in train.at[i, 'DistanciaMeta'] or 'cuello' in train.at[i, 'DistanciaMeta']):
#       train.at[i, 'DistanciaMeta'] = '0.3'
#     if(('CABEZA') in train.at[i, 'DistanciaMeta'] or ('Cabeza') in train.at[i, 'DistanciaMeta'] or ('cabeza') in train.at[i, 'DistanciaMeta']):
#       train.at[i, 'DistanciaMeta'] = '0.2'
#     if(('NARIZ') in train.at[i, 'DistanciaMeta'] or ('Nariz') in train.at[i, 'DistanciaMeta'] or ('nariz') in train.at[i, 'DistanciaMeta']):
#       train.at[i, 'DistanciaMeta'] = '0.1'
#     if(('EMPATE') in train.at[i, 'DistanciaMeta'] or ('Empate') in train.at[i, 'DistanciaMeta'] or ('empate') in train.at[i, 'DistanciaMeta'] or train.at[i, 'DistanciaMeta'] == '-'):
#       train.at[i, 'DistanciaMeta'] = '0'
#     if(('CABALLO') in train.at[i, 'DistanciaMeta'] or ('Caballo') in train.at[i, 'DistanciaMeta'] or ('caballo') in train.at[i, 'DistanciaMeta']):
#       train.at[i, 'DistanciaMeta'] = '1'
#     if((' C') in train.at[i, 'DistanciaMeta']):
#       train.at[i, 'DistanciaMeta'] = train.at[i, 'DistanciaMeta'].replace(' C', '')
#     if(train.at[i, 'DistanciaMeta'] == '1/4'):
#       train.at[i, 'DistanciaMeta'] = '0.25'
#     if(train.at[i, 'DistanciaMeta'] == '1/2'):
#       train.at[i, 'DistanciaMeta'] = '0.5'
#     if(train.at[i, 'DistanciaMeta'] == '3/4'):
#       train.at[i, 'DistanciaMeta'] = '0.75'
#     if((' 1/4') in train.at[i, 'DistanciaMeta']):
#       train.at[i, 'DistanciaMeta'] = train.at[i, 'DistanciaMeta'].replace(' 1/4', '')
#       train.at[i, 'DistanciaMeta'] = train.at[i, 'DistanciaMeta'] + '.25'
#     if((' 1/2') in train.at[i, 'DistanciaMeta']):
#       train.at[i, 'DistanciaMeta'] = train.at[i, 'DistanciaMeta'].replace(' 1/2', '')
#       train.at[i, 'DistanciaMeta'] = train.at[i, 'DistanciaMeta'] + '.5'
#     if((' 3/4') in train.at[i, 'DistanciaMeta']):
#       train.at[i, 'DistanciaMeta'] = train.at[i, 'DistanciaMeta'].replace(' 3/4', '')
#       train.at[i, 'DistanciaMeta'] = train.at[i, 'DistanciaMeta'] + '.75'

# conversionDistanciaMetaTrain();
# train['DistanciaMeta'] = train['DistanciaMeta'].astype('float')
# train['DistanciaMeta'] = np.where(train['DistanciaMeta'] > 20, 20, train['DistanciaMeta'])
# train['DistanciaMeta'].describe()


## **Tratamiento (Peso)**

In [ ]:
# train['Peso'] = train['Peso'].str.partition('-')[0]
# train['Peso'] = train['Peso'].str[:2]

In [ ]:
train['Peso'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Fecha)**


In [ ]:
import datetime as dt
from datetime import date


def switchMonth(month):
  if month == 'ENERO':
   return '01'
  elif month == 'FEBRERO':
    return '02'
  elif month == 'MARZO':
   return '03'
  elif month == 'ABRIL':
    return '04'
  elif month == 'MAYO':
    return '05'
  elif month == 'JUNIO':
    return '06'
  elif month == 'JULIO':
    return '07'
  elif month == 'AGOSTO':
    return '08'
  elif month == 'SEPTIEMBRE':
    return '09'
  elif month == 'OCTUBRE':
     return '10'
  elif month == 'NOVIEMBRE':
    return '11'
  elif month == 'DICIEMBRE':
    return '12'

def convertDate(fechas):
  index = 0
  for element in fechas:
    sum = 0
    year = element[-4:]
    dateAux = year
    day = element[:2]
    if(int(day) < 10):
      day = "0" + day[:1]
    month = element[5:]
    month = ''.join(month.split())[:-6].upper()
    month = switchMonth(month)
    dateAux = dateAux + "-" + month + "-" + day

    today = date.today()
    yearToday = int(str(today)[:4])
    monthToday = int(str(today)[5:7])
    dayToday = int(str(today)[8:10])
    if(yearToday > int(year)):
      sum += (yearToday - int(year))*365
    sum += (monthToday - int(month))*30
    sum += (dayToday - int(day))
    datetimeAux = dt.datetime(int(year), int(month), int(day))
    train.at[index, 'FechaAux'] = datetimeAux

    train.at[index, 'year'] = int(year)
    train.at[index, 'month'] = int(month)
    train.at[index, 'day'] = int(day)
    calculateSeason(int(month), index)

    # train.at[index, 'DiasDesdeCarrera'] = sum
    index += 1

def calculateSeason(month, index):
  # train['Invierno'] = 0
  # train['Primavera'] = 0
  # train['Verano'] = 0
  train['Otoño'] = 0
  # if(month >= 12 and month <=2):
  #    train.at[index, 'Invierno'] = month
  # if(month >= 3 and month <=5):
  #    train.at[index, 'Primavera'] = month
  # if(month >= 6 and month <=8):
  #    train.at[index, 'Verano'] = month
  if(month >= 9 and month <=11):
     train.at[index, 'Otoño'] = month




t_array = train["Fecha"]
convertDate(t_array)
train['FechaAux'].head(70)

In [ ]:
train.head(5)

In [ ]:
from datetime import datetime

def daysSincePreviousRace():
  for index, row in train.iterrows():

    nombreCaballo = row['NombreCaballo']
    dict = {}
    conta = 0
    for indice, fila in trainOg.iterrows():
      if( fila['NombreCaballo'] == nombreCaballo):
        rowData = fila['FechaAux'].date()
        dict[rowData] = indice
        conta += 1

    listaFechasAux = list(filter(lambda n: n < row['FechaAux'].date(), list(dict.keys())))
    if(listaFechasAux):
      mostRecentRace = max(listaFechasAux)
      jockeyRecent = trainOg.at[dict[mostRecentRace], 'Jinete']
      daysDifference = row['FechaAux'].date() - mostRecentRace
      train.at[index, 'DaysSincePreviousRace'] = daysDifference.days


      if(jockeyRecent == row['Jinete']):
         train.at[index, 'SameJockey'] = 1
      else:
        train.at[index, 'SameJockey'] = 0
    else:
        train.at[index, 'SameJockey'] = 0
        train.at[index, 'DaysSincePreviousRace'] = 45



    # Rellenar dictionario con keys que sean las fechas en las que ha participado el caballo y los values , el index en el que aparecen
    # Deberias buscar primero las fechas anteriores en las que corrio un caballo antes de este codigo
    # y en caso de que no haya fechas hacer algo al respecto
    # En caso de qie si haya , por keys.values y coger anteriores y la mas reciente. Obtener key.value de jinete y comprobar con jinete actual .
    # Rambien calcular dias de diferencia desde fecha


daysSincePreviousRace()
train.info()

In [ ]:
train.head()

## Calculo Contricantes
---

In [ ]:
def calculoContrincantes(listaF, listaH, longitud):
  for fecha in listaF:
    for horario in listaH:
      index = 0
      listaIndex = []
      sum = 0
      while(index < longitud):
        if((train.at[index, 'Fecha'] == fecha ) and (train.at[index, 'Hora'] == horario )):
          listaIndex.append(index)
          sum += 1
        index += 1
      for indice in listaIndex:
        train.at[indice, 'Contrincantes'] = sum


listaFechas = train['Fecha'].unique().tolist()
listaHorarios = train['Hora'].unique().tolist()
longitud = len(train.index)

calculoContrincantes(listaFechas, listaHorarios, longitud)




## **Busqueda Caballos Ganadores en ciertas condiciones**
---

### Busqueda UltimasActuacionesLargo
---

In [ ]:
from datetime import timedelta

def ultimasActuacionesIter(indices, indicesOg):
  for i in range(indices):
    print(i)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0.0
    sum = 0.0

    sum_odds = 0.0
    media_odds = 0.0

    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      nombreAux= trainOg.at[j, 'NombreCaballo']
      sinceDate = fecha - timedelta(days=200)
      recentDates = fecha - timedelta(days = 30)
      if((nombreAux == nombre) and (fechaAux < fecha) and (sinceDate < fechaAux)):
        print("Valor Puesto-->", trainPuesto.at[j, 'PuestoOg'], "/", fechaAux)
        value = 0.0
        value = (trainPuesto.at[j, 'PuestoOg'] )
        media += value
        sum += 1
        # if(recentDates < fechaAux):
          # value = value**(1.5);
        odds = 0.0
        odds = (trainPuesto.at[j, 'Dividendo'])
        media_odds += odds
        sum_odds += 1

    print("Suma Medias -->", media, "/", sum, "<-- Nº Act Anteriores")
    if(sum != 0):
      media /= sum
    if(sum == 0):
      media = (train.at[I, 'Contrincantes'] / 2)
      train.at[i, 'UltimasActuacionesLargo'] = media
    else:
      train.at[i, 'UltimasActuacionesLargo'] = media
    print(i, "-->", nombre, " MediaUltAct->", media)

    #Dividendo
    if(sum_odds != 0):
      media_odds /= sum_odds
    if(sum_odds == 0):
      media_odds = 5.69
      train.at[i, 'UltimosDividendosLargo'] = media_odds
    else:
      train.at[i, 'UltimosDividendosLargo'] = media_odds


indicesOg = len(trainOg.index)
indices = len(train.index)
ultimasActuacionesIter(indices, indicesOg)

In [ ]:
train['UltimasActuacionesLargo'][:20].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda UltimasActuacionesMedio
---

In [ ]:
from datetime import timedelta

def ultimasActuacionesIter(indices, indicesOg):
  for i in range(indices):
    print(i)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0.0
    sum = 0.0

    media_odds = 0.0
    sum_odds = 0.0

    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      nombreAux= trainOg.at[j, 'NombreCaballo']
      sinceDate = fecha - timedelta(days=120)
      recentDates = fecha - timedelta(days = 30)
      if((nombreAux == nombre) and (fechaAux < fecha) and (sinceDate < fechaAux)):
        print("Valor Puesto-->", trainPuesto.at[j, 'PuestoOg'], "/", fechaAux)
        value = 0.0
        value = (trainPuesto.at[j, 'PuestoOg'] )
        media += value
        sum += 1
        # if(recentDates < fechaAux):
          # value = value**(1.4);
        odds = 0.0
        odds = (trainPuesto.at[j, 'Dividendo'])
        media_odds += odds
        sum_odds += 1

    print("Suma Medias -->", media, "/", sum, "<-- Nº Act Anteriores")
    if(sum != 0):
      media /= sum
    if(sum == 0):
      if( train.at[i, 'UltimasActuacionesLargo'] != 2.6):
        media =  train.at[i, 'UltimasActuacionesLargo']
      else:
        media = 2.6
      train.at[i, 'UltimasActuacionesMedio'] = media
    else:
      train.at[i, 'UltimasActuacionesMedio'] = media
    print(i, "-->", nombre, " MediaUltAct->", media)

    #Dividendo
    if(sum_odds != 0):
      media_odds /= sum_odds
    if(sum_odds == 0):
      if( train.at[i, 'UltimosDividendosLargo'] != 5.69):
        media_odds =  train.at[i, 'UltimosDividendosLargo']
      else:
        media_odds = 5.69
      train.at[i, 'UltimosDividendosMedio'] = media_odds
    else:
      train.at[i, 'UltimosDividendosMedio'] = media_odds


indicesOg = len(trainOg.index)
indices = len(train.index)
ultimasActuacionesIter(indices, indicesOg)

In [ ]:
train['UltimasActuacionesMedio'][:20].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda UltimasActuacionesCorto
---

In [ ]:
from datetime import timedelta

def ultimasActuacionesIter(indices, indicesOg):
  for i in range(indices):
    print(i)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0.0
    sum = 0.0

    media_odds = 0.0
    sum_odds = 0.0

    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      nombreAux= trainOg.at[j, 'NombreCaballo']
      sinceDate = fecha - timedelta(days=90)
      recentDates = fecha - timedelta(days = 22)
      if((nombreAux == nombre) and (fechaAux < fecha) and (sinceDate < fechaAux)):
        print("Valor Puesto-->", trainPuesto.at[j, 'PuestoOg'], "/", fechaAux)
        value = 0.0
        value = (trainPuesto.at[j, 'PuestoOg'] )
        media += value
        sum += 1
        # if(recentDates < fechaAux):
          # value = value**(1.3);

        odds = 0.0
        odds = (trainPuesto.at[j, 'Dividendo'])
        media_odds += odds
        sum_odds += 1

    print("Suma Medias -->", media, "/", sum, "<-- Nº Act Anteriores")
    if(sum != 0):
      media /= sum
    if(sum == 0):
      if( train.at[i, 'UltimasActuacionesMedio'] == 2.6):
        if( train.at[i, 'UltimasActuacionesLargo'] != 2.6):
          media =  train.at[i, 'UltimasActuacionesLargo']
        else:
          media = 2.6
      else:
        media =  train.at[i, 'UltimasActuacionesMedio']
      train.at[i, 'UltimasActuacionesCorto'] = media
    else:
      train.at[i, 'UltimasActuacionesCorto'] = media
    print(i, "-->", nombre, " MediaUltAct->", media)

    #Dividendo
    if(sum_odds != 0):
      media_odds /= sum_odds
    if(sum_odds == 0):
      if( train.at[i, 'UltimosDividendosMedio'] == 5.69):
        if( train.at[i, 'UltimosDividendosLargo'] != 5.69):
          media_odds =  train.at[i, 'UltimosDividendosLargo']
        else:
          media_odds = 5.69
      else:
        media_odds =  train.at[i, 'UltimosDividendosMedio']
      train.at[i, 'UltimosDividendosCorto'] = media_odds
    else:
      train.at[i, 'UltimosDividendosCorto'] = media_odds


indicesOg = len(trainOg.index)
indices = len(train.index)
ultimasActuacionesIter(indices, indicesOg)

In [ ]:
train['UltimasActuacionesCorto'][:20].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3


### Busqueda UltimasDistancias
---

In [ ]:
from datetime import timedelta

def ultimasDistanciasCorto(indices, indicesOg):
  for i in range(indices):
    print(i)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0.0
    sum = 0.0

    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      nombreAux= trainOg.at[j, 'NombreCaballo']
      sinceDate = fecha - timedelta(days=60)
      if((nombreAux == nombre) and (fechaAux < fecha) and (sinceDate < fechaAux)):
        print("Valor Distancia-->", trainOg.at[j, 'DistanciaMeta'], "/", fechaAux)
        value = 0.0
        value = (trainOg.at[j, 'DistanciaMeta'] )
        media += value
        sum += 1
    print("Suma Medias -->", media, "/", sum, "<-- Nº Act Anteriores")
    if(sum != 0):
      media /= sum
    if(sum == 0):
      media = 3.5
      train.at[i, 'ultimasDistanciasCorto'] = media
    else:
      train.at[i, 'ultimasDistanciasCorto'] = media
    print(i, "-->", nombre, " MediaUltAct->", media)


indicesOg = len(trainOg.index)
indices = len(train.index)
ultimasDistanciasCorto(indices, indicesOg)

In [ ]:
from datetime import timedelta

def ultimasDistanciasMedio(indices, indicesOg):
  for i in range(indices):
    print(i)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0.0
    sum = 0.0

    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      nombreAux= trainDistancia.at[j, 'NombreCaballo']
      sinceDate = fecha - timedelta(days=120)
      if((nombreAux == nombre) and (fechaAux < fecha) and (sinceDate < fechaAux)):
        print("Valor Distancia-->", trainOg.at[j, 'DistanciaMeta'], "/", fechaAux)
        value = 0.0
        value = (trainOg.at[j, 'DistanciaMeta'] )
        media += value
        sum += 1
    print("Suma Medias -->", media, "/", sum, "<-- Nº Act Anteriores")
    if(sum != 0):
      media /= sum
    if(sum == 0):
      media = 3.5
      train.at[i, 'ultimasDistanciasMedio'] = media
    else:
      train.at[i, 'ultimasDistanciasMedio'] = media
    print(i, "-->", nombre, " MediaUltAct->", media)


indicesOg = len(trainOg.index)
indices = len(train.index)
ultimasDistanciasMedio(indices, indicesOg)

In [ ]:
from datetime import timedelta

def ultimasDistanciasLargo(indices, indicesOg):
  for i in range(indices):
    print(i)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0.0
    sum = 0.0

    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      nombreAux= trainOg.at[j, 'NombreCaballo']
      sinceDate = fecha - timedelta(days=200)
      if((nombreAux == nombre) and (fechaAux < fecha) and (sinceDate < fechaAux)):
        print("Valor Distancia-->", trainOg.at[j, 'DistanciaMeta'], "/", fechaAux)
        value = 0.0
        value = (trainOg.at[j, 'DistanciaMeta'] )
        media += value
        sum += 1
    print("Suma Medias -->", media, "/", sum, "<-- Nº Act Anteriores")
    if(sum != 0):
      media /= sum
    if(sum == 0):
      media = 3.5
      train.at[i, 'ultimasDistanciasLargo'] = media
    else:
      train.at[i, 'ultimasDistanciasLargo'] = media
    print(i, "-->", nombre, " MediaUltAct->", media)


indicesOg = len(trainOg.index)
indices = len(train.index)
ultimasDistanciasLargo(indices, indicesOg)

In [ ]:
train = train.drop('DistanciaMeta', axis=1)
trainOg = trainOg.drop('DistanciaMeta', axis=1)


### Busqueda Distancia
---

In [ ]:
pd.set_option('display.max_rows', None)
train.head(70)

In [ ]:
def comprobacionTipoDistanciaTrain(indice):
  tipo = 0   # 0, 1, 2 / Corta, Media, Larga
  val = trainOg.at[indice, 'Distancia']
  if(val >= 3200):
    tipo = 2
  if((val >= 1600) & (val <= 2400)):
    tipo = 1
  return tipo;

def comprobacionTipoDistanciaTest(indice):
  tipo = 0   # 0, 1, 2 / Corta, Media, Larga
  val = train.at[indice, 'Distancia']
  if(val >= 3200):
    tipo = 2
  if((val >= 1600) & (val <= 2400)):
    tipo = 1
  return tipo;

def destrezaDistanciaTest(indices,indicesOg):
  for i in range(indices):
    tipo = comprobacionTipoDistanciaTest(i)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0
    sumAux = 0
    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      tipoAux = comprobacionTipoDistanciaTrain(j)
      nombreAux= trainOg.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (trainPuesto.at[j, 'PuestoOg'])
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (trainPuesto.at[j, 'PuestoOg'])
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/") # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones en esas condiciones y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaDistancia'] = 1
    else:
      train.at[i, 'DestrezaDistancia'] = 0

    #IMP! Valorar si es buena idea crear una columna a partir de la variable "media" + "mediaAux" obtenida aqui.

indices = len(train.index)
indicesOg = len(trainOg.index)
destrezaDistanciaTest(indices, indicesOg)



#Len Intervalos
print(len(train[(train['Distancia'] < 1600).tolist()]))
print(len(train[((train['Distancia'] >= 1600) & (train['Distancia'] <= 2400) ).tolist()]))
print(len(train[(train['Distancia'] >= 3200).tolist()]))

In [ ]:
pd.set_option('display.max_rows', None)
train[['NombreCaballo','DestrezaDistancia']].head(64)

### Busqueda Meteorologia
---

In [ ]:
train['Meteorología'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoMeteorologia(indice, listaMeteo):
  tipo = -1 # 0 -> Lluvia / 1 -> Nublado / 2 -> Soleado, Despejado
  val = train.at[indice, 'Meteorología']
  for meteo in listaMeteo:
   if("Lluvia" in val or "Llovizna" in val or "precipitaciones" in val or "lluvias" in val):
      tipo = 1
   elif("Soleado" in val or "Despejado" in val):
      tipo = 2
   else:
     tipo = 0
  return tipo;

def comprobacionTipoMeteorologiaOg(indice, listaMeteo):
  tipo = -1 # 0 -> Lluvia / 1 -> Nublado / 2 -> Soleado, Despejado
  val = trainOg.at[indice, 'Meteorología']
  for meteo in listaMeteo:
   if("Lluvia" in val or "Llovizna" in val or "precipitaciones" in val or "lluvias" in val):
      tipo = 1
   elif("Soleado" in val or "Despejado" in val):
      tipo = 2
   else:
     tipo = 0
  return tipo;

def destrezaMeteorologia(indices, indicesOg, listaMeteo):
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoMeteorologia(i, listaMeteo)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0
    sumAux = 0
    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      tipoAux = comprobacionTipoMeteorologiaOg(j, listaMeteo)
      nombreAux= trainOg.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (trainPuesto.at[j, 'PuestoOg'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (trainPuesto.at[j, 'PuestoOg'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/") # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaMeteo'] = 1
      print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
    else:
      train.at[i, 'DestrezaMeteo'] = 0

indicesOg = len(trainOg.index)
indices = len(train.index)
listaMeteo = train['Meteorología'].unique().tolist()
destrezaMeteorologia(indices, indicesOg,  listaMeteo)

### Busqueda Lluvia
---

In [ ]:
train['LLuvia'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoLLuvia(indice, listaLLuvia):
  tipo = -1 # 0 -> 0.0 / 1 -> > 0.0 / 2 -> > 1
  val = train.at[indice, 'LLuvia']
  for meteo in listaMeteo:
   if(val > 0):
      tipo = 1
   elif(val > 1):
      tipo = 2
   else:
     tipo = 0
  return tipo;


def comprobacionTipoLLuviaOg(indice, listaLLuvia):
  tipo = -1 # 0 -> 0.0 / 1 -> > 0.0 / 2 -> > 1
  val = trainOg.at[indice, 'LLuvia']
  for meteo in listaMeteo:
   if(val > 0):
      tipo = 1
   elif(val > 1):
      tipo = 2
   else:
     tipo = 0
  return tipo;

def destrezaLLuvia(indices, indicesOg,  listaLLuvia):
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoLLuvia(i, listaLLuvia)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0
    sumAux = 0
    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      tipoAux = comprobacionTipoLLuviaOg(j, listaLLuvia)
      nombreAux= trainOg.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (trainPuesto.at[j, 'PuestoOg'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (trainPuesto.at[j, 'PuestoOg'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/")
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaLLuvia'] = 1
    else:
      train.at[i, 'DestrezaLLuvia'] = 0

indicesOg= len(trainOg.index)
indices = len(train.index)
listaLLuvia = train['LLuvia'].unique().tolist()
destrezaLLuvia(indices, indicesOg,  listaLLuvia)

In [ ]:
train['DestrezaLLuvia'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda Temperatura
---

In [ ]:
train['Temperatura'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoTemperatura(indice, listaTemperatura):
  tipo = -1 # 0 -> < 20 / 1 -> > 30 > x < 20 / 2 -> > > 30
  val = train.at[indice, 'Temperatura']
  for meteo in listaMeteo:
   if(val > 30):
      tipo = 2
   elif(val < 20):
      tipo = 0
   else:
     tipo = 0
  return tipo;

def comprobacionTipoTemperaturaOg(indice, listaTemperatura):
  tipo = -1 # 0 -> < 20 / 1 -> > 30 > x < 20 / 2 -> > > 30
  val = trainOg.at[indice, 'Temperatura']
  for meteo in listaMeteo:
   if(val > 30):
      tipo = 2
   elif(val < 20):
      tipo = 0
   else:
     tipo = 0
  return tipo;

def destrezaTemperatura(indices, indicesOg, listaTemperatura, ):
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoTemperatura(i, listaTemperatura)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0
    sumAux = 0
    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      tipoAux = comprobacionTipoTemperaturaOg(j, listaTemperatura)
      nombreAux= trainOg.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (trainPuesto.at[j, 'PuestoOg'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (trainPuesto.at[j, 'PuestoOg'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/")
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaTemperatura'] = 1
    else:
      train.at[i, 'DestrezaTemperatura'] = 0

indicesOg = len(trainOg.index)
indices = len(train.index)
listaTemperatura = train['Temperatura'].unique().tolist()
destrezaTemperatura(indices, indicesOg,  listaTemperatura)

In [ ]:
train['DestrezaTemperatura'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda Viento
---

In [ ]:
train['Viento'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoViento(indice, listaViento):
  tipo = -1 # 0 -> < 5 / 1 -> > 30 > x > 5 / 2 -> > > 30
  val = train.at[indice, 'Viento']
  for meteo in listaMeteo:
   if(val > 30):
      tipo = 2
   elif(val < 5):
      tipo = 0
   else:
     tipo = 0
  return tipo;


def comprobacionTipoVientoOg(indice, listaViento):
  tipo = -1 # 0 -> < 5 / 1 -> > 30 > x > 5 / 2 -> > > 30
  val = trainOg.at[indice, 'Viento']
  for meteo in listaMeteo:
   if(val > 30):
      tipo = 2
   elif(val < 5):
      tipo = 0
   else:
     tipo = 0
  return tipo;

def destrezaViento(indices, indicesOg, listaViento):
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoViento(i, listaViento)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0
    sumAux = 0
    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      tipoAux = comprobacionTipoVientoOg(j, listaViento)
      nombreAux= trainOg.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (trainPuesto.at[j, 'PuestoOg'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (trainPuesto.at[j, 'PuestoOg'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/")
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaViento'] = 1
    else:
      train.at[i, 'DestrezaViento'] = 0

indicesOg =  len(trainOg.index)
indices = len(train.index)
listaViento = train['Viento'].unique().tolist()
destrezaViento(indices,indicesOg, listaViento)

In [ ]:
train['DestrezaViento'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda Terreno
---

In [ ]:
train['Terreno'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoTerreno(indice, listaTerreno):
  tipo = -1 # 0 -> Arena / 1 -> H - Blando / 2 -> H - Bueno
  val = train.at[indice, 'Terreno']
  if(val == "H-Bueno"):
    element = element.replace('H-Bueno', 'H - Bueno')
    train.at[indice, 'Terreno'] = val
  for meteo in listaMeteo:
   if(val == 'H - Bueno'):
      tipo = 2
   elif(val == 'H - Blando'):
      tipo = 1
   elif(val == 'H - Blando'):
      tipo = 0
   else:
      tipo = 4
  return tipo;

def comprobacionTipoTerrenoOg(indice, listaTerreno):
  tipo = -1 # 0 -> Arena / 1 -> H - Blando / 2 -> H - Bueno
  val = trainOg.at[indice, 'Terreno']
  if(val == "H-Bueno"):
    element = element.replace('H-Bueno', 'H - Bueno')
    trainOg.at[indice, 'Terreno'] = val
  for meteo in listaMeteo:
   if(val == 'H - Bueno'):
      tipo = 2
   elif(val == 'H - Blando'):
      tipo = 1
   elif(val == 'H - Blando'):
      tipo = 0
   else:
      tipo = 4
  return tipo;

def destrezaTerreno(indices,indicesOg, listaTerreno):
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoTerreno(i, listaTerreno)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0
    sumAux = 0
    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      tipoAux = comprobacionTipoTerrenoOg(j, listaTerreno)
      nombreAux= trainOg.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (trainPuesto.at[j, 'PuestoOg'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (trainPuesto.at[j, 'PuestoOg'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/")
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaTerreno'] = 1
    else:
      train.at[i, 'DestrezaTerreno'] = 0


indices = len(train.index)
indicesOg = len(trainOg.index)
listaTerreno = train['Terreno'].unique().tolist()
destrezaTerreno(indices,indicesOg,  listaTerreno)

In [ ]:
train['DestrezaTerreno'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

### **Tratamiento (CaballoDistintos)**

In [ ]:
train.loc[~train["NombreCaballo"].isin(listAparicionesHorse), "NombreCaballo"] = "OtherHorse" # Lo posponemos a este momento(por ahora)

## **Tratamiento (Edad)**

In [ ]:
train['Edad'].value_counts()[:10].sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Mantilla)**

In [ ]:
train['Mantilla'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Problemas)**


In [ ]:
train['Problemas'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
train['Problemas'] = train['Problemas'].fillna('0')
train['Problema_Nulo'] = np.where(train['Problemas'].str.contains('0'),1,0)
train['Problema_1'] = np.where(train['Problemas'].str.contains('1'),1,0)
train['Problema_2'] = np.where(train['Problemas'].str.contains('2'),1,0)
train['Problema_3'] = np.where(train['Problemas'].str.contains('3'),1,0)
train['Problema_4'] = np.where(train['Problemas'].str.contains('4'),1,0)
train['Problema_5'] = np.where(train['Problemas'].str.contains('5'),1,0)
train['Problema_6'] = np.where(train['Problemas'].str.contains('6'),1,0)
train['Problema_7'] = np.where(train['Problemas'].str.contains('7'),1,0)
train['Problema_8'] = np.where(train['Problemas'].str.contains('8'),1,0)
train.head()



## **Tratamiento (UltimasActuaciones)**


In [ ]:
# def count_lastRaces(arrayLast):
#   index = 0
#   for element in arrayLast:
#     element = element.replace(',', '')
#     element = element.replace('[', '')
#     element = element[:len(element)-1]
#     sum = 0
#     if 'Debutante' in element:
#       sum = 10
#       element = []
#     else:
#       element = element.replace('NP', '08')
#       element = element.replace('Desc', '08')
#       element = element.replace(' ', '')
#       i = len(element)
#       j = len(element)
#       iter = (len(element)) / 2
#       count = 0
#       while(iter > 0):
#         if(count == 0):
#           i = i-2
#           sum += int(element[i:j])**1.3
#           j = j-2
#         elif(count == 1):
#           i = i-2
#           sum += int(element[i:j])**1.2
#           j = j-2
#         else:
#           print(iter)
#           i = i-2
#           sum += int(element[i:j])
#           j = j-2
#         count += 1
#         iter -= 1
#       sum = sum / len(element)
#       if(index < 30):
#         print('******INDEX ', index+2, '*****')
#         print('Sum -->', sum, '/Len -->', len(element))
#         print('Result -->', sum, '/Element -->' ,element)
#     train.at[index, 'MediaUltimasActuaciones'] = sum
#     train.at[index, 'CantidadActuaciones'] = len(element)
#     index += 1





# t_array = train["UltimasActuaciones"]
# count_lastRaces(t_array)

In [ ]:
#  train['MediaUltimasActuaciones'].head(25)

## **Tratamiento (Hora)**


In [ ]:
train['Hora'] = train['Hora'].str.replace(':', '')

## **Tratamiento (Terreno)**


## **Tratamiento (Distancia)**


## **Tratamiento (Tipo)**


## **Tratamiento (SentidoHipodromo)**


## **Tratamiento (Meteorologia)**


## **Tratamiento (Lluvia)**


## **Tratamiento (Viento)**


## **Tratamiento (TemperaturaMax)**


## **Tratamiento (TemperaturaMin)**


## **Tratamiento (Hipodromo)**


#Continuacion

In [ ]:
def conversion():
  for i in range(len(train.index)):
    if(train.at[i ,'Contrincantes'] < 11):
      if(train.at[i ,'PuestoOg'] == 1 or train.at[i, 'PuestoOg'] == 2):
        train.at[i, 'Puesto'] = 0
      else:
        train.at[i, 'Puesto'] = 1
    else:
      if(train.at[i, 'PuestoOg'] == 1 or train.at[i, 'PuestoOg'] == 2 or train.at[i, 'PuestoOg'] == 3):
         train.at[i, 'Puesto'] = 0
      else:
        train.at[i, 'Puesto'] = 1

conversion()

In [ ]:
y_testNN = train.pop('Puesto')

In [ ]:
X_val = train
ids = train
X_val = X_val.reset_index()
X_val.info()
X_valTF = X_val #TensorFlow

In [ ]:
column_transformer_test = sklearn.compose.ColumnTransformer(transformers=[
    ("drop", "drop", ['Fecha','year', 'day', 'Problemas','Hora', 'FechaAux', 'Otoño', 'month', 'Distancia', 'Mantilla', 'Terreno', 'Tipo', 'Meteorología', 'LLuvia', 'Viento', 'Temperatura', 'Hipodromo', 'Contrincantes', 'PuestoOg']),
    ("scale", sklearn.preprocessing.StandardScaler(), []),
    # ("num", sklearn.preprocessing.MinMaxScaler(), ['Distancia', 'Edad', 'Peso']),
    ("one-hot", sklearn.preprocessing.OneHotEncoder(handle_unknown="ignore"), ['NombreCaballo','Propietario', 'Preparador', 'Jinete']),
], remainder='passthrough');

# column_transformer = sklearn.compose.ColumnTransformer(transformers=[
#     ("drop", "drop", ['UltimasActuaciones','Fecha','year','day', 'Problemas']),
#     ("scale", sklearn.preprocessing.StandardScaler(), []),
#     ("one-hot", sklearn.preprocessing.OneHotEncoder(handle_unknown="ignore"), ['NombreCaballo','Peso','Edad', 'Mantilla','Propietario', 'Preparador', 'Jinete', 'estacion', 'FechaAux', 'Terreno','Distancia', 'Tipo', 'Meteorología', 'LLuvia', 'Viento', 'TemperaturaMax', 'TemperaturaMin', 'Hipodromo']),
# ], remainder='passthrough');

In [ ]:
train = train.reset_index()
X_testAux1NN = X_val

X_testArrayNN = pd.DataFrame(column_transformer_test.fit_transform(X_val).toarray())
X_val = column_transformer_test.fit_transform(X_val)



In [ ]:
X_val

In [ ]:
column_transformer_train.get_feature_names_out()

In [ ]:
feat_names = column_transformer_train.get_feature_names_out()

In [ ]:
column_transformer_test.get_feature_names_out()

In [ ]:
print(X_val)

In [ ]:
X_testNN = X_train
X_testAuxNN = X_val

# Algoritmos
---

In [ ]:
from sklearn.metrics import mean_absolute_error
tree.fit(X, y);

print('---')

print('Tree (train): ' + str(mean_absolute_error(y_train, tree.predict(X_train))))
predicted_yTree = tree.predict(X_val)
print('Tree (test): ' + str(mean_absolute_error(y_testNN, tree.predict(X_val))))

print('---')

print('RF (train): ' + str(mean_absolute_error(y_train, rf.predict(X_train))))
predicted_yRF = rf.predict(X_val)
print('RF (test): ' + str(mean_absolute_error(y_testNN, rf.predict(X_val))))

print('---')

print('XGB (train): ' + str(mean_absolute_error(y_train, xgb_model.predict(X_train))))
predicted_yXGB = xgb_model.predict(X_val)
print('XGB (test): ' + str(mean_absolute_error(y_testNN, xgb_model.predict(X_val))))

print('---')

print('Cat (train): ' + str(mean_absolute_error(y_train, model_cat_tun.predict(X_train))))
predicted_yCat = model_cat_tun.predict(X_val)
print('Cat (test): ' + str(mean_absolute_error(y_testNN, model_cat_tun.predict(X_val))))

print('---')

print('Lineal (train): ' + str(mean_absolute_error(y_train, linear.predict(X_train))))
predicted_yLineal = linear.predict(X_val)
print('Lineal (test): ' + str(mean_absolute_error(y_testNN, linear.predict(X_val))))



print('---')

print('Ridge (train): ' + str(mean_absolute_error(y_train, ridge.predict(X_train))))
predicted_yRidge = ridge.predict(X_val)
print('Ridge (test): ' + str(mean_absolute_error(y_testNN, ridge.predict(X_val))))


print('---')

print('LASSO (train): ' + str(mean_absolute_error(y_train, lasso.predict(X_train))))
predicted_yLASSO = lasso.predict(X_val)
print('LASSO (test): ' + str(mean_absolute_error(y_testNN, lasso.predict(X_val))))


print('---')

print('ElasticNet (train): ' + str(mean_absolute_error(y_train, elasticnet.predict(X_train))))
predicted_yElastic = elasticnet.predict(X_val)
print('ElasticNet (test): ' + str(mean_absolute_error(y_testNN, elasticnet.predict(X_val))))

In [ ]:
from matplotlib.pyplot import figure
feat_importances = pd.Series(xgb_model.feature_importances_, index=feat_names).sort_values(ascending = False)
plt.figure(figsize=(10,30))
feat_importances.plot(kind = 'barh')

In [ ]:
def get_xgb_imp(xgb, feat_names):
    from numpy import array
    imp_vals = xgb.booster().get_fscore()
    imp_dict = {feat_names[i]:float(imp_vals.get('f'+str(i),0.)) for i in range(len(feat_names))}
    total = array(imp_dict.values()).sum()
    return {k:v/total for k,v in imp_dict.items()}

In [ ]:
import graphviz
from sklearn.tree import export_graphviz

dot_data = export_graphviz(tree, out_file=None)
graph = graphviz.Source(dot_data)

In [ ]:
trainOg = train
train['Prediccion'] = 0
train['Puesto'] = 0
train['Carrera'] = 0
y_val = pd.concat([train['Carrera'], train['Puesto'],train['NombreCaballo'], train['Prediccion'], train['UltimasActuacionesCorto'], train['UltimasActuacionesMedio'],train['UltimasActuacionesLargo'],train['DestrezaDistancia'], train['SameJockey'], train['FechaAux'], train['Hora']],axis=1)

In [ ]:
def ordenarTest(horarios, index, x):
  carrera = 0
  aux = -1
  for horario in horarios:
    if(aux != horario):
      carrera += 1
      aux = horario
    dict = {}
    conta = 0
    for i in index:
      horarioActual = x.at[i, 'Hora']
      if(horario == horarioActual ):
          x.at[i, 'Carrera'] = carrera
          prediccion = x.at[i, 'Prediccion']
          dict[i] = prediccion
          conta += 1
    puesto = 1
    for w in sorted(dict, key=dict.get):
      x.at[w, 'Puesto'] = puesto
      puesto += 1

def ordenarTestReverse(horarios, index, x):
  carrera = 0
  aux = -1
  for horario in horarios:
    if(aux != horario):
      carrera += 1
      aux = horario
    dict = {}
    conta = 0
    for i in index:
      horarioActual = x.at[i, 'Hora']
      if(horario == horarioActual ):
          x.at[i, 'Carrera'] = carrera
          prediccion = x.at[i, 'Prediccion']
          dict[i] = prediccion
          conta += 1
    puesto = 1
    for w in sorted(dict, key=dict.get, reverse=True):
      x.at[w, 'Puesto'] = puesto
      puesto += 1



horarios = train['Hora'].tolist()
index = train['index'].tolist()

In [ ]:
y_valTree = y_val
y_valRF = y_val
y_valXgb = y_val
y_valCat = y_val
y_val1 = y_val
y_val2= y_val
y_val3 = y_val
y_val4= y_val
y_valNN = y_val
y_valTreeXgb = y_val

In [ ]:
y_valTree['Prediccion'] = predicted_yTree
xTree = y_valTree
ordenarTestReverse(horarios, index, xTree)
xTree.to_csv('outputTree.csv' , index=False)

In [ ]:
y_valRF['Prediccion'] = predicted_yRF
xRF = y_valRF
ordenarTestReverse(horarios, index, xRF)
xRF.to_csv('outputRF.csv' , index=False)

In [ ]:
y_valXgb['Prediccion'] = predicted_yXGB
xXGB = y_valXgb
ordenarTestReverse(horarios, index, xXGB)
xXGB.to_csv('outputXgb.csv' , index=False)


In [ ]:
y_valTreeXgb['Prediccion'] = (predicted_yTree + predicted_yXGB)
xTreeXgb = y_valTreeXgb
ordenarTestReverse(horarios, index, xTreeXgb)
xTreeXgb.to_csv('outputTreeXGB.csv' , index=False)

In [ ]:
y_valCat['Prediccion'] = predicted_yCat
xCat = y_valCat
ordenarTestReverse(horarios, index, xCat)
xCat.to_csv('outputCat.csv' , index=False)

In [ ]:

y_val1['Prediccion'] = predicted_yLineal
x1 = y_val1
ordenarTestReverse(horarios, index, x1)
x1.to_csv('outputLineal.csv' , index=False)
y_val2['Prediccion'] = predicted_yRidge
# print(x2)
# x2 = y_val2
# ordenarTestReverse(horarios, index, x2)
# x2.to_csv('outputRidge.csv', index=False)
y_val3['Prediccion'] = predicted_yLASSO
x3 = y_val3
ordenarTestReverse(horarios, index, x3)
x3.to_csv('outputLASSO.csv', index=False)
y_val4['Prediccion'] = predicted_yElastic
x4 = y_val4
ordenarTestReverse(horarios, index, x4)
x4.to_csv('outputElastic.csv' , index=False)

- **Considero importante realizar una explicacion en cada subida de .csv sobre en que se diferencia respecto al anterior**

# NN
---

**Convertir 20% de la tabla superior en valores 1 ( en carrera de 11 corredores , puestos 1 , 2 y 3 y en carrera de 10 corredores , puestos 1 y 2. Resto de corredores dar valor de 0.**

In [ ]:
import tensorflow as tf
import numpy as np


np.random.seed(42) # Establecer la semilla aleatoria
tf.random.set_seed(42) # Establecer la semilla aleatoria en TensorFlow

model = tf.keras.Sequential([
    tf.keras.layers.Dense(162, activation='relu', input_shape=(189,)),
    tf.keras.layers.Dense(81, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.005),
              loss='mse',
              metrics=['mae', 'mse'])
print(model.summary())

In [ ]:
print((X_trainArrayNN.shape))
print((y_trainNN.shape))

print(X_testArrayNN.shape)
print(y_testNN.shape)

# print((y_trainNN))
# print((X_testArrayNN))

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X_trainArrayNN.values, y_trainNN.values))
train_dataset = dataset.shuffle(len(X_trainArrayNN)).batch(500)
dataset = tf.data.Dataset.from_tensor_slices((X_testArrayNN.values, y_testNN.values))
validation_dataset = dataset.shuffle(len(X_testArrayNN)).batch(500)

class EarlyStopping(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    '''
    Stops training when 0.2 > val_loss is reached
    '''
    # Get the current accuracy and check if it is above 95%
    if(logs.get('val_loss') < 0.2):

      # Stop training if condition is met
      print("\nThreshold reached. Stopping training...")
      self.model.stop_training = True

# Let's instantiate our ES class
early_stopping = EarlyStopping()

# early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', start_from_epoch=50, patience = 50, mode='min')

print("Start training..\n")
history = model.fit(train_dataset, epochs=200 , validation_data=validation_dataset, callbacks=[early_stopping])
# history = model.fit(train_dataset, epochs=1000 , validation_data=validation_dataset, callbacks=[early_stop])

print("Done.")

# precision = history.history['precision']
# val_precision = history.history['val_precision']
# loss = history.history['loss']
# val_loss = history.history['val_loss']
# epochs = range(1, len(precision) + 1)

# plt.plot(epochs, precision, 'b', label='Training precision')
# plt.plot(epochs, val_precision, 'r', label='Validation precision')
# plt.title('Training and validation precision')
# plt.legend()
# plt.figure()

# plt.plot(epochs, loss, 'b', label='Training loss')
# plt.plot(epochs, val_loss, 'r', label='Validation loss')
# plt.title('Training and validation loss')
# plt.legend()
# plt.show()

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.ylim([0,1])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.ylim([0,1])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_loss'],
           label = 'Val Error')
  plt.ylim([0,1])
  plt.legend()

  plt.show()


plot_history(history)

In [ ]:
loss, mae, mse = model.evaluate(X_testArrayNN, y_testNN, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))

In [ ]:
test_predictions = model.predict(X_testArrayNN).flatten()

plt.scatter(y_testNN, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])

In [ ]:
error = test_predictions - y_testNN
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [MPG]")
_ = plt.ylabel("Count")

In [ ]:
predicted_yNN = model.predict(X_val)
y_valNN['Prediccion'] = predicted_yNN
xNN = y_valNN
ordenarTestReverse(horarios, index, xNN)
xNN.to_csv('outputNN.csv' , index=False)

predicted_yNN

In [ ]:
print('Neural Network (test): ' + str(mean_absolute_error(y_testNN, model.predict(X_val))))